In [1]:
# Celda 0 — Config (versión dinámica)
from pathlib import Path
import pandas as pd, numpy as np, math, re, os, json
from datetime import datetime

# -------- Raíz de outputs del proyecto --------
OUT = Path(os.environ.get("OUT_DIR", "/home/cesar/proyectos/TFM_SNN/outputs")).resolve()

# -------- Filtros "paper set" (por defecto) --------
# Puedes sobreescribirlos con env vars: PRESET_KEEP, ENCODER_KEEP, SEED_KEEP, METHODS_KEEP
def _parse_set(env_name, default):
    val = os.environ.get(env_name, None)
    if val is None:
        return default
    if val.strip().lower() in {"none", "", "all"}:
        return None
    return set([v.strip() for v in val.split(",") if v.strip()])

PRESET_KEEP   = _parse_set("PRESET_KEEP",   {"fast"})   # None → no filtra por preset
ENCODER_KEEP  = _parse_set("ENCODER_KEEP",  {"rate"})   # None → no filtra
SEED_KEEP     = _parse_set("SEED_KEEP",     None)       # p.ej. "42,7"
METHODS_KEEP  = _parse_set("METHODS_KEEP",  None)

ONLY_NEW_RUNNER = os.environ.get("ONLY_NEW_RUNNER", "1") not in {"0","false","False"}

# Ventana temporal: SINCE="2025-11-11" o SINCE="3d" (últimos 3 días). Si no se da, no filtra por fecha.
SINCE = os.environ.get("SINCE", "").strip()
def _compute_mtime_from(since_str: str):
    if not since_str:
        return None
    try:
        if since_str.lower().endswith("d"):
            days = int(since_str[:-1])
            return pd.Timestamp.now().normalize() - pd.Timedelta(days=days)
        return pd.Timestamp(since_str)
    except Exception:
        return None
MTIME_FROM = _compute_mtime_from(SINCE) or None

# -------- Comparabilidad "dura" --------
STRICT_CFG = os.environ.get("STRICT_CFG", "1") not in {"0","false","False"}

# -------- Métrica compuesta --------
ALPHA_COMPOSITE = float(os.environ.get("ALPHA_COMPOSITE", "0.5"))

# -------- Opciones de informe / narrativa --------
# IGNORE_NAIVE_IN_REPORTS = os.environ.get("IGNORE_NAIVE", "0") in {"1","true","True"}
GROUP_BY_FULL_METHOD    = os.environ.get("GROUP_BY_FULL_METHOD", "0") in {"1","true","True"}
# RELATIVE_BASELINE       = os.environ.get("RELATIVE_BASELINE", "ewc")  # "naive","ewc","auto", o "" para desactivar
# BASELINE_MATCH_STRICT   = os.environ.get("BASELINE_MATCH_STRICT", "1") not in {"0","false","False"}

IGNORE_NAIVE_IN_REPORTS = True       # excluye naive de tablas/plots de informe
RELATIVE_BASELINE = "ewc"            # baseline para comparativas
BASELINE_MATCH_STRICT = True         # emparejamiento estricto de runs comparables

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)

# -------- Carpeta SUMMARY dinámica --------
def _token(val):
    if val is None: return "all"
    if isinstance(val, (set, list, tuple)):
        return "-".join(sorted(map(str, val)))
    return str(val)

now = pd.Timestamp.now(tz="Europe/Madrid")
auto_label = f"paperset_{_token(PRESET_KEEP)}_{_token(ENCODER_KEEP)}_{now.strftime('%Y-%m-%d_%H%M')}"
SUMMARY_LABEL = os.environ.get("SUMMARY_LABEL", "").strip() or auto_label

SUMMARY = (OUT / "summary" / SUMMARY_LABEL).resolve()
SUMMARY.mkdir(parents=True, exist_ok=True)
print(f"[SUMMARY] {SUMMARY}")

# Symlink "latest" → esta ejecución (si el FS lo permite)
LATEST = OUT / "summary" / "latest"
try:
    if LATEST.exists() or LATEST.is_symlink():
        LATEST.unlink()
    LATEST.symlink_to(SUMMARY, target_is_directory=True)
except Exception:
    pass

# Guardar manifest de filtros/ajustes para trazabilidad
manifest = {
    "summary_label": SUMMARY_LABEL,
    "created_at": now.isoformat(),
    "OUT": str(OUT),
    "filters": {
        "PRESET_KEEP": sorted(list(PRESET_KEEP)) if PRESET_KEEP else None,
        "ENCODER_KEEP": sorted(list(ENCODER_KEEP)) if ENCODER_KEEP else None,
        "SEED_KEEP": sorted(list(SEED_KEEP)) if SEED_KEEP else None,
        "METHODS_KEEP": sorted(list(METHODS_KEEP)) if METHODS_KEEP else None,
        "ONLY_NEW_RUNNER": ONLY_NEW_RUNNER,
        "MTIME_FROM": str(MTIME_FROM) if MTIME_FROM is not None else None,
    },
    "comparability": {"STRICT_CFG": STRICT_CFG},
    "composite": {"ALPHA_COMPOSITE": ALPHA_COMPOSITE},
    "report_opts": {
        "IGNORE_NAIVE_IN_REPORTS": IGNORE_NAIVE_IN_REPORTS,
        "GROUP_BY_FULL_METHOD": GROUP_BY_FULL_METHOD,
        "RELATIVE_BASELINE": RELATIVE_BASELINE,
        "BASELINE_MATCH_STRICT": BASELINE_MATCH_STRICT,
    },
}
(SUMMARY / "summary_manifest.json").write_text(json.dumps(manifest, indent=2), encoding="utf-8")

# --- Manifest "último" para reutilizar rutas en otras celdas/notebooks
LAST_PATH = OUT / "summary" / "_last.json"

def _last_update(**kwargs):
    # Carga el estado actual (si existe)
    try:
        cur = json.loads(LAST_PATH.read_text(encoding="utf-8"))
    except Exception:
        cur = {}

    # Campos fijos útiles
    cur.update({
        "summary_label": SUMMARY_LABEL,
        "summary_dir": str(SUMMARY.resolve()),
        "created_at": now.isoformat(),
    })

    # Normaliza y añade las nuevas rutas/etiquetas
    for k, v in list(kwargs.items()):
        if isinstance(v, (str, Path)):
            kwargs[k] = str(Path(v).resolve())
    cur.update(kwargs)

    LAST_PATH.write_text(json.dumps(cur, indent=2), encoding="utf-8")
    print(f"[LAST] actualizado → {LAST_PATH}")

# Crea/actualiza _last.json al inicio con la info básica de esta ejecución
_last_update()


[SUMMARY] /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427
[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


In [2]:
# Celda 1 — Utils

def _safe_float(x, default=np.nan):
    try:
        if x is None: return default
        if isinstance(x, (int, float)): return float(x)
        s = str(x).strip()
        if s.lower() in {"nan","none","null",""}: return default
        return float(s)
    except Exception:
        return default

def _read_json(p: Path):
    try:
        if p.exists():
            return json.loads(p.read_text(encoding="utf-8"))
    except Exception:
        pass
    return None

def _read_csv_df(p: Path):
    try:
        if p.exists():
            return pd.read_csv(p)
    except Exception:
        pass
    return None

def _abs_run_dir(rel: str|Path) -> Path:
    rel = str(rel)
    return (OUT / rel) if not rel.startswith(str(OUT)) else Path(rel)

def run_mtime(rel: str|Path) -> float:
    rd = _abs_run_dir(rel)
    mt = 0.0
    for root, _, files in os.walk(rd):
        for f in files:
            try:
                mt = max(mt, (Path(root)/f).stat().st_mtime)
            except Exception:
                pass
    return mt

def canonical_method(m: str) -> str:
    if not m: return "none"
    m = m.lower()
    if m.startswith("ewc"):        return "ewc"
    if m.startswith("as-snn"):     return "as-snn"
    if m.startswith("sa-snn"):     return "sa-snn"
    if m.startswith("sca-snn"):    return "sca-snn"
    if m.startswith("rehearsal"):  return "rehearsal"
    if m.startswith("naive"):      return "naive"
    return m

def _read_forgetting(run_dir: Path) -> dict:
    """
    Lee primero forgetting_summary.json (preferente).
    Si no existe, lee forgetting.json (anidado por tareas) y calcula la media.
    Devuelve claves normalizadas: circuito1_forget_abs/rel, circuito2_forget_abs/rel, avg_forget_rel.
    """
    def _nanmean2(a, b):
        vals = [x for x in [a, b] if x is not None and not math.isnan(_safe_float(x))]
        return float(np.mean([_safe_float(x) for x in vals])) if vals else np.nan

    # 1) summary (preferido)
    summ = _read_json(run_dir / "forgetting_summary.json")
    if isinstance(summ, dict):
        c1_abs = _safe_float(summ.get("circuito1_forget_abs", summ.get("task_1_forget_abs")))
        c1_rel = _safe_float(summ.get("circuito1_forget_rel", summ.get("task_1_forget_rel")))
        c2_abs = _safe_float(summ.get("circuito2_forget_abs", summ.get("task_2_forget_abs")))
        c2_rel = _safe_float(summ.get("circuito2_forget_rel", summ.get("task_2_forget_rel")))
        avg_rel = _safe_float(summ.get("avg_forget_rel", summ.get("avg_forgetting_rel")))
        if math.isnan(avg_rel):
            avg_rel = _nanmean2(c1_rel, c2_rel)
        return {
            "circuito1_forget_abs": c1_abs,
            "circuito1_forget_rel": c1_rel,
            "circuito2_forget_abs": c2_abs,
            "circuito2_forget_rel": c2_rel,
            "avg_forget_rel":       avg_rel,
        }

    # 2) forgetting.json (anidado por tareas) o plano
    js = _read_json(run_dir / "forgetting.json") or {}
    if not isinstance(js, dict):
        return {}

    def pick(d, *keys):
        for k in keys:
            if k in d: return d[k]
        return None

    c1_abs = _safe_float(pick(js, "circuito1_forget_abs","task_1_circuito1_forget_abs","c1_forget_abs","task_1_forget_abs"))
    c1_rel = _safe_float(pick(js, "circuito1_forget_rel","task_1_circuito1_forget_rel","c1_forget_rel","task_1_forget_rel"))
    c2_abs = _safe_float(pick(js, "circuito2_forget_abs","task_2_circuito2_forget_abs","c2_forget_abs","task_2_forget_abs"))
    c2_rel = _safe_float(pick(js, "circuito2_forget_rel","task_2_circuito2_forget_rel","c2_forget_rel","task_2_forget_rel"))
    avg_rel = _safe_float(pick(js, "avg_forget_rel","avg_forgetting_rel","mean_forget_rel","forget_rel_avg"))

    # Si sigue faltando, leer anidado por tarea
    if ("circuito1" in js) and isinstance(js["circuito1"], dict):
        if math.isnan(c1_abs): c1_abs = _safe_float(js["circuito1"].get("forget_abs"))
        if math.isnan(c1_rel): c1_rel = _safe_float(js["circuito1"].get("forget_rel"))
    if ("circuito2" in js) and isinstance(js["circuito2"], dict):
        if math.isnan(c2_abs): c2_abs = _safe_float(js["circuito2"].get("forget_abs"))
        if math.isnan(c2_rel): c2_rel = _safe_float(js["circuito2"].get("forget_rel"))

    if math.isnan(avg_rel):
        avg_rel = _nanmean2(c1_rel, c2_rel)

    return {
        "circuito1_forget_abs": c1_abs,
        "circuito1_forget_rel": c1_rel,
        "circuito2_forget_abs": c2_abs,
        "circuito2_forget_rel": c2_rel,
        "avg_forget_rel":       avg_rel,
    }


In [3]:
# Celda 2 — Reconstrucción 100% desde ficheros (robusta a formatos)

import os, re, json, math
from pathlib import Path
import numpy as np
import pandas as pd

# ——— Fallback por si _read_forgetting no quedó definido en la Celda 1 ———
try:
    _read_forgetting
except NameError:
    def _read_forgetting(run_dir: Path):
        def _nanmean2(a, b):
            vals = [x for x in [a, b] if x is not None and not math.isnan(_safe_float(x))]
            return (float(np.mean([_safe_float(x) for x in vals])) if vals else np.nan)
        summ = _read_json(run_dir / "forgetting_summary.json")
        if isinstance(summ, dict):
            c1_abs = _safe_float(summ.get("circuito1_forget_abs", summ.get("task_1_forget_abs")))
            c1_rel = _safe_float(summ.get("circuito1_forget_rel", summ.get("task_1_forget_rel")))
            c2_abs = _safe_float(summ.get("circuito2_forget_abs", summ.get("task_2_forget_abs")))
            c2_rel = _safe_float(summ.get("circuito2_forget_rel", summ.get("task_2_forget_rel")))
            avg_rel = _safe_float(summ.get("avg_forget_rel", summ.get("avg_forgetting_rel")))
            if math.isnan(avg_rel): avg_rel = _nanmean2(c1_rel, c2_rel)
            return {
                "circuito1_forget_abs": c1_abs, "circuito1_forget_rel": c1_rel,
                "circuito2_forget_abs": c2_abs, "circuito2_forget_rel": c2_rel,
                "avg_forget_rel": avg_rel,
            }
        js = _read_json(run_dir / "forgetting.json") or {}
        if not isinstance(js, dict): return {}
        def pick(d, *keys):
            for k in keys:
                if k in d: return d[k]
            return None
        c1_abs = _safe_float(pick(js, "circuito1_forget_abs","task_1_circuito1_forget_abs","c1_forget_abs","task_1_forget_abs"))
        c1_rel = _safe_float(pick(js, "circuito1_forget_rel","task_1_circuito1_forget_rel","c1_forget_rel","task_1_forget_rel"))
        c2_abs = _safe_float(pick(js, "circuito2_forget_abs","task_2_circuito2_forget_abs","c2_forget_abs","task_2_forget_abs"))
        c2_rel = _safe_float(pick(js, "circuito2_forget_rel","task_2_circuito2_forget_rel","c2_forget_rel","task_2_forget_rel"))
        avg_rel = _safe_float(pick(js, "avg_forget_rel","avg_forgetting_rel","mean_forget_rel","forget_rel_avg"))
        if ("circuito1" in js) and isinstance(js["circuito1"], dict):
            if math.isnan(c1_abs): c1_abs = _safe_float(js["circuito1"].get("forget_abs"))
            if math.isnan(c1_rel): c1_rel = _safe_float(js["circuito1"].get("forget_rel"))
        if ("circuito2" in js) and isinstance(js["circuito2"], dict):
            if math.isnan(c2_abs): c2_abs = _safe_float(js["circuito2"].get("forget_abs"))
            if math.isnan(c2_rel): c2_rel = _safe_float(js["circuito2"].get("forget_rel"))
        if math.isnan(avg_rel):
            vals = [x for x in [c1_rel, c2_rel] if not math.isnan(_safe_float(x))]
            avg_rel = float(np.mean([_safe_float(x) for x in vals])) if vals else np.nan
        return {
            "circuito1_forget_abs": c1_abs, "circuito1_forget_rel": c1_rel,
            "circuito2_forget_abs": c2_abs, "circuito2_forget_rel": c2_rel,
            "avg_forget_rel": avg_rel,
        }

# ——— Helpers internos ———

def _parse_basic_meta(run_dir: Path) -> dict:
    """Extrae preset, method, encoder, model, seed, T, amp, batch_size desde los artefactos.
       Prioridad: run_row.json → manifest de tarea → heurística de nombre de carpeta.
    """
    jrow = _read_json(run_dir / "run_row.json") or {}

    def gj(*ks, default=None):
        obj = jrow
        for k in ks:
            if not isinstance(obj, dict) or (k not in obj):
                return default
            obj = obj[k]
        return obj

    # Campos directos de run_row.json (top-level o dentro de meta/data/training)
    preset   = jrow.get("preset") or gj("meta","preset")
    method   = jrow.get("method") or gj("meta","method")
    encoder  = jrow.get("encoder") or gj("meta","encoder")
    model    = jrow.get("model")   or jrow.get("model_name") or gj("meta","model") or gj("meta","model_name")
    seed     = jrow.get("seed")    or gj("meta","seed")
    T        = jrow.get("T")       or gj("data","T") or gj("meta","data","T") or gj("meta","T")
    amp      = jrow.get("amp")     or gj("training","amp") or gj("meta","amp")
    batch_sz = jrow.get("batch_size") or gj("meta","batch_size")

    # Manifest de la primera tarea (por si faltan cosas)
    man1 = _read_json(run_dir / "task_1_circuito1" / "manifest.json")
    if isinstance(man1, dict):
        meta1 = man1.get("meta", {}) if isinstance(man1.get("meta", {}), dict) else {}
        if not model:        model     = meta1.get("model") or man1.get("model") or man1.get("model_name")
        if batch_sz is None: batch_sz  = meta1.get("batch_size", batch_sz)
        # Claves típicas en tu formato:
        if T is None:        T         = meta1.get("T", T)        # T vive en meta
        if amp is None:      amp       = meta1.get("amp", amp)    # amp vive en meta
        if not encoder:      encoder   = meta1.get("encoder", encoder)
        if not preset:       preset    = meta1.get("preset", preset)
        if not method:       method    = meta1.get("method", method)

    # Heurística de nombre de carpeta si faltan preset/method/encoder
    if not preset or not method or not encoder:
        name = run_dir.name
        m = re.match(r"continual_([^_]+)_([^_].*?)_(rate|latency|raw|image).*", name)
        if m:
            preset  = preset  or m.group(1)
            method  = method  or m.group(2)
            encoder = encoder or m.group(3)

    # Tipados/normalizaciones
    seed    = _safe_float(seed)
    T       = _safe_float(T)
    if isinstance(amp, str):
        amp = amp.strip().lower() in {"true","1","yes","y"}
    elif isinstance(amp, (int, float)):
        amp = bool(amp)
    elif amp is not None and not isinstance(amp, bool):
        amp = None
    batch_sz = _safe_float(batch_sz)

    return dict(
        preset=preset, method=method, encoder=encoder, model=model,
        seed=seed, T=T, amp=amp, batch_size=batch_sz
    )

def _read_per_task_perf(run_dir: Path) -> dict:
    """Devuelve dict por tarea con {'best_mae','final_mae'} normalizadas.
       Soporta per_task_perf.json (lista/dict) y per_task_perf.csv.
       Normaliza nombres de tarea (lower) y fusiona JSON+CSV para rellenar huecos.
    """
    def _norm_row_dictlike(d):
        tname = (str(d.get("task_name") or d.get("task") or d.get("name") or "")).strip().lower()
        # añadimos 'test_mae' como candidato de 'best'
        best_candidates  = ["best_mae","val_best_mae","best","mae_best","min_mae","test_mae"]
        final_candidates = ["final_mae","val_final_mae","val_last_mae","last_mae","mae_last","mae_final"]
        best  = next((d.get(k) for k in best_candidates  if k in d), None)
        final = next((d.get(k) for k in final_candidates if k in d), None)
        return tname, {"best_mae": _safe_float(best), "final_mae": _safe_float(final)}

    out = {}

    # 1) JSON
    js = _read_json(run_dir / "per_task_perf.json")
    if js is not None:
        if isinstance(js, list):
            for row in js:
                if isinstance(row, dict):
                    t, val = _norm_row_dictlike(row)
                    if t:
                        out[t] = val
        elif isinstance(js, dict):
            if all(isinstance(v, dict) for v in js.values()):
                for k, v in js.items():
                    t, val = _norm_row_dictlike({"task_name": k, **v})
                    if t:
                        out[t] = val
            else:
                try:
                    df = pd.DataFrame(js)
                    for _, row in df.iterrows():
                        t, val = _norm_row_dictlike(row.to_dict())
                        if t:
                            out[t] = val
                except Exception:
                    pass

    # 2) CSV (fusiona y rellena NaNs si los hay)
    df = _read_csv_df(run_dir / "per_task_perf.csv")
    if df is not None and not df.empty:
        for _, row in df.iterrows():
            tn = (str(row.get("task_name") or row.get("task") or row.get("name") or "")).strip().lower()
            if not tn:
                continue
            cur = out.get(tn, {"best_mae": np.nan, "final_mae": np.nan})
            if math.isnan(_safe_float(cur.get("best_mae"))):
                cur["best_mae"] = _safe_float(row.get("val_best_mae", row.get("best_mae", row.get("test_mae"))))
            if math.isnan(_safe_float(cur.get("final_mae"))):
                cur["final_mae"] = _safe_float(row.get("val_last_mae", row.get("val_final_mae", row.get("final_mae"))))
            out[tn] = cur

    return out

def _read_efficiency(run_dir: Path):
    """Lee emisiones/tiempo desde efficiency_summary.json; si falta, intenta emissions.csv y run_row.json."""
    j = _read_json(run_dir / "efficiency_summary.json") or {}
    emissions = _safe_float(j.get("emissions_kg"), default=np.nan)
    elapsed   = _safe_float(j.get("elapsed_sec"),   default=np.nan)

    if math.isnan(emissions):
        df = _read_csv_df(run_dir / "emissions.csv")
        if df is not None:
            col = "co2e_kg" if "co2e_kg" in df.columns else ("emissions_kg" if "emissions_kg" in df.columns else None)
            if col:
                emissions = float(df[col].sum())

    if math.isnan(elapsed) or math.isnan(emissions):
        rj = _read_json(run_dir / "run_row.json") or {}
        if math.isnan(elapsed):
            elapsed = _safe_float(rj.get("elapsed_sec"), default=elapsed)
        if math.isnan(emissions):
            emissions = _safe_float(rj.get("emissions_kg"), default=emissions)

    return emissions, elapsed

def _read_eval_matrix(run_dir: Path) -> pd.DataFrame | None:
    """Carga eval_matrix (csv preferente) o reconstruye desde json {'tasks':[], 'mae_matrix':[[]]}."""
    p_csv = run_dir / "eval_matrix.csv"
    if p_csv.exists():
        try:
            return pd.read_csv(p_csv)
        except Exception:
            pass

    js = _read_json(run_dir / "eval_matrix.json")
    if isinstance(js, dict) and ("tasks" in js) and ("mae_matrix" in js):
        tasks = list(js.get("tasks") or [])
        mat   = js.get("mae_matrix") or []
        # Intentamos formato de columnas como en CSV: 'task', 'after_circuito1', 'after_circuito2',...
        try:
            mat = np.array(mat, dtype=float)
            cols = ["task"] + [f"after_{t}" for t in tasks]
            data = {"task": tasks}
            for j, cname in enumerate(cols[1:]):
                colvals = [row[j] if j < len(row) else np.nan for row in mat]
                data[cname] = colvals
            return pd.DataFrame(data)
        except Exception:
            # fallback genérico
            try:
                return pd.DataFrame(js)
            except Exception:
                return None
    return None

def _compute_best_final_from_eval(eval_df: pd.DataFrame, task_token: str):
    """Obtiene BEST/FINAL mirando primero por FILAS (col 'task') y,
    si no existe, hace fallback a columnas que contengan el token."""
    if eval_df is None or eval_df.empty:
        return (np.nan, np.nan)

    # 1) Preferir formato fila: 'task' == task_token
    tcol = None
    for c in eval_df.columns:
        if str(c).strip().lower() == "task":
            tcol = c
            break
    if tcol is not None:
        mask = eval_df[tcol].astype(str).str.lower() == str(task_token).lower()
        if mask.any():
            row = eval_df.loc[mask].iloc[0]
            data_cols = [c for c in eval_df.columns if c != tcol]

            vals = pd.to_numeric(row[data_cols], errors="coerce").values.astype(float)
            finite = np.isfinite(vals)
            best = float(np.nanmin(vals)) if finite.any() else np.nan

            # FINAL = valor en la última columna temporal (p.ej., after_última_tarea)
            final = _safe_float(row[data_cols[-1]])
            return (best, final)

    # 2) Fallback: columnas que contengan el token (formatos antiguos)
    cols = [c for c in eval_df.columns if str(task_token).lower() in str(c).lower()]
    if cols:
        dfc = eval_df[cols].apply(pd.to_numeric, errors="coerce")
        vals = dfc.values.astype(float)
        finite = np.isfinite(vals)
        best = float(np.nanmin(vals)) if finite.any() else np.nan
        final = _safe_float(dfc.iloc[-1, -1])
        return (best, final)

    return (np.nan, np.nan)


def _compute_forgetting_from_eval_matrix(eval_df: pd.DataFrame, per_task: dict):
    """Olvido para c1 tras aprender c2. C2 no olvida (0)."""
    out = {}
    t1_keys = [k for k in (per_task or {}).keys() if "circuito1" in str(k).lower()]
    best_t1 = None
    if t1_keys:
        best_t1 = _safe_float((per_task.get(t1_keys[0]) or {}).get("best_mae"))
    if best_t1 is None or math.isnan(best_t1):
        best_t1, _ = _compute_best_final_from_eval(eval_df, "circuito1")

    _, final_t1_after_last = _compute_best_final_from_eval(eval_df, "circuito1")
    if best_t1 is None or math.isnan(best_t1) or final_t1_after_last is None or math.isnan(final_t1_after_last):
        return out

    forget_abs = max(0.0, final_t1_after_last - best_t1)
    forget_rel = forget_abs / max(1e-9, best_t1)
    out["circuito1_forget_abs"] = forget_abs
    out["circuito1_forget_rel"] = forget_rel
    out["circuito2_forget_abs"] = 0.0
    out["circuito2_forget_rel"] = 0.0
    out["avg_forget_rel"] = forget_rel
    return out

def _read_continual_results(run_dir: Path) -> dict:
    """Fallback final para MAEs desde continual_results.json."""
    j = _read_json(run_dir / "continual_results.json")
    if not isinstance(j, dict):
        return {}
    c1 = j.get("circuito1", {}) or {}
    c2 = j.get("circuito2", {}) or {}
    return {
        "c1_best":  _safe_float(c1.get("test_mae")),
        "c1_final": _safe_float(c1.get("after_circuito2_mae")),
        "c2_best":  _safe_float(c2.get("test_mae")),
        "c2_final": _safe_float(c2.get("test_mae")),
    }

def build_results_table_from_disk(base_out: Path) -> pd.DataFrame:
    rows = []
    run_dirs = [p for p in base_out.glob("continual_*") if p.is_dir()]
    print(f"[INFO] Escaneando {len(run_dirs)} runs en {base_out}")

    for rd in run_dirs:
        meta      = _parse_basic_meta(rd)
        per_task  = _read_per_task_perf(rd)
        eff_kg, elapsed = _read_efficiency(rd)
        forget_js = _read_forgetting(rd) or {}
        eval_df   = _read_eval_matrix(rd)

        # MAEs por tarea con fallback a eval_matrix y, si hace falta, continual_results.json
        c1_best = c1_final = c2_best = c2_final = np.nan

        t1_keys = [k for k in per_task.keys() if "circuito1" in str(k).lower()]
        if t1_keys:
            c1_best  = _safe_float(per_task[t1_keys[0]].get("best_mae"),  default=np.nan)
            c1_final = _safe_float(per_task[t1_keys[0]].get("final_mae"), default=np.nan)
        if math.isnan(c1_best) or math.isnan(c1_final):
            b, f = _compute_best_final_from_eval(eval_df, "circuito1")
            if math.isnan(c1_best):  c1_best  = b
            if math.isnan(c1_final): c1_final = f

        t2_keys = [k for k in per_task.keys() if "circuito2" in str(k).lower()]
        if t2_keys:
            c2_best  = _safe_float(per_task[t2_keys[0]].get("best_mae"),  default=np.nan)
            c2_final = _safe_float(per_task[t2_keys[0]].get("final_mae"), default=np.nan)
        if math.isnan(c2_best) or math.isnan(c2_final):
            b, f = _compute_best_final_from_eval(eval_df, "circuito2")
            if math.isnan(c2_best):  c2_best  = b
            if math.isnan(c2_final): c2_final = f

        # Fallback definitivo: continual_results.json
        if any(math.isnan(x) for x in [c1_best, c1_final, c2_best, c2_final]):
            cr = _read_continual_results(rd)
            if math.isnan(c1_best):  c1_best  = _safe_float(cr.get("c1_best"),  default=c1_best)
            if math.isnan(c1_final): c1_final = _safe_float(cr.get("c1_final"), default=c1_final)
            if math.isnan(c2_best):  c2_best  = _safe_float(cr.get("c2_best"),  default=c2_best)
            if math.isnan(c2_final): c2_final = _safe_float(cr.get("c2_final"), default=c2_final)

        # Olvido (summary/json) o cálculo desde eval_matrix
        f_c1_abs = _safe_float(forget_js.get("circuito1_forget_abs"))
        f_c1_rel = _safe_float(forget_js.get("circuito1_forget_rel"))
        f_c2_abs = _safe_float(forget_js.get("circuito2_forget_abs"))
        f_c2_rel = _safe_float(forget_js.get("circuito2_forget_rel"))
        avg_f_rel = _safe_float(forget_js.get("avg_forget_rel"))
        if all(math.isnan(x) for x in [f_c1_abs, f_c1_rel, f_c2_abs, f_c2_rel, avg_f_rel]):
            comp = _compute_forgetting_from_eval_matrix(eval_df, per_task)
            if comp:
                f_c1_abs = comp.get("circuito1_forget_abs", f_c1_abs)
                f_c1_rel = comp.get("circuito1_forget_rel", f_c1_rel)
                f_c2_abs = comp.get("circuito2_forget_abs", f_c2_abs)
                f_c2_rel = comp.get("circuito2_forget_rel", f_c2_rel)
                avg_f_rel = comp.get("avg_forget_rel", avg_f_rel)

        row = dict(
            run_dir=str(rd.relative_to(base_out)),
            preset=meta["preset"],
            method=meta["method"],
            encoder=meta["encoder"],
            model=meta["model"],
            seed=meta["seed"],
            T=meta["T"],
            batch_size=meta["batch_size"],
            amp=meta["amp"],
            emissions_kg=eff_kg,
            elapsed_sec=elapsed,
            circuito1_best_mae=c1_best,
            circuito1_final_mae=c1_final,
            circuito2_best_mae=c2_best,
            circuito2_final_mae=c2_final,
            circuito1_forget_abs=f_c1_abs,
            circuito1_forget_rel=f_c1_rel,
            circuito2_forget_abs=f_c2_abs,
            circuito2_forget_rel=f_c2_rel,
            avg_forget_rel=avg_f_rel,
        )
        rows.append(row)

    df = pd.DataFrame(rows)

    # Flags extra
    df["is_new_runner"] = df["run_dir"].apply(lambda rd: (_abs_run_dir(rd) / "run_row.json").exists() or (_abs_run_dir(rd) / "run_row.csv").exists())
    df["mtime"] = df["run_dir"].apply(run_mtime)
    df["mtime_dt"] = pd.to_datetime(df["mtime"], unit="s")
    df["method_base"] = df["method"].astype(str).apply(canonical_method)

    # Tipado numérico: EXCLUYE 'amp' (lo normalizamos como boolean más abajo)
    numeric_cols = [
        "seed","T","batch_size","emissions_kg","elapsed_sec",
        "circuito1_best_mae","circuito1_final_mae","circuito2_best_mae","circuito2_final_mae",
        "circuito1_forget_abs","circuito1_forget_rel","circuito2_forget_abs","circuito2_forget_rel","avg_forget_rel"
    ]
    for c in numeric_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # 'amp' como boolean (permite NA)
    if "amp" in df.columns:
        if df["amp"].dtype == bool:
            df["amp"] = df["amp"].astype("boolean")
        else:
            df["amp"] = df["amp"].map(
                lambda v: bool(v) if isinstance(v, (bool,int,float))
                else (str(v).strip().lower() in {"true","1","yes","y"} if isinstance(v, str) else pd.NA)
            ).astype("boolean")

    out_csv = SUMMARY / "results_table_fromdisk.csv"
    df.to_csv(out_csv, index=False)
    print(f"[OK] results_table_fromdisk → {out_csv} | filas:", len(df))
    return df


In [4]:
# Celda 3 — Construcción + diagnóstico NaNs

df_all = build_results_table_from_disk(OUT)
display(df_all.head(3))

nan_cols = ["circuito1_best_mae","circuito1_final_mae","circuito2_best_mae","circuito2_final_mae","avg_forget_rel"]
print("[DEBUG] NaNs globales:", {c:int(df_all[c].isna().sum()) for c in nan_cols if c in df_all.columns})


[INFO] Escaneando 246 runs en /home/cesar/proyectos/TFM_SNN/outputs
[OK] results_table_fromdisk → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/results_table_fromdisk.csv | filas: 246


,run_dir,preset,method,encoder,model,seed,T,batch_size,amp,emissions_kg,elapsed_sec,circuito1_best_mae,circuito1_final_mae,circuito2_best_mae,circuito2_final_mae,circuito1_forget_abs,circuito1_forget_rel,circuito2_forget_abs,circuito2_forget_rel,avg_forget_rel,is_new_runner,mtime,mtime_dt,method_base
0,continual_fast_as-snn_gr_0.6_lam_0.36_att_f6_f...,fast,as-snn_gr_0.6_lam_0.36_att_f6,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.005630,771.873337,0.189499,0.262805,0.190483,0.190483,0.073306,0.386843,0.0,0.0,0.193422,True,1.763134e+09,2025-11-14 15:27:35.800247431,as-snn
1,continual_fast_ewc_lam_5e+07_fast_ewc_l5e7_fb3...,fast,ewc_lam_5e+07,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.008511,1921.457078,0.140196,0.231559,0.199714,0.199714,0.091363,0.651682,0.0,0.0,0.325841,True,1.762962e+09,2025-11-12 15:38:42.516785860,ewc
2,continual_fast_as-snn_gr_0.6_lam_0.4_att_f6_fa...,fast,as-snn_gr_0.6_lam_0.4_att_f6,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.004914,770.022141,0.127710,0.175707,0.165940,0.165940,0.047996,0.375820,0.0,0.0,0.187910,True,1.763093e+09,2025-11-14 04:09:08.899888992,as-snn


[DEBUG] NaNs globales: {'circuito1_best_mae': 9, 'circuito1_final_mae': 9, 'circuito2_best_mae': 9, 'circuito2_final_mae': 9, 'avg_forget_rel': 9}


In [5]:
# Celda 4 — Selección y comparabilidad
def _filter_paperset(df: pd.DataFrame, require_both_tasks: bool = True) -> pd.DataFrame:
    m = df.copy()
    if PRESET_KEEP:
        m = m[m["preset"].isin(PRESET_KEEP)]
    if ENCODER_KEEP:
        m = m[m["encoder"].isin(ENCODER_KEEP)]
    if SEED_KEEP:
        m = m[m["seed"].isin(SEED_KEEP)]
    if METHODS_KEEP:
        m = m[m["method_base"].isin(METHODS_KEEP)]
    if ONLY_NEW_RUNNER:
        m = m[m["is_new_runner"] == True]
    if MTIME_FROM is not None:
        m = m[m["mtime_dt"] >= MTIME_FROM]

    # Runs con ambas tareas evaluadas (mejor y final). Evita arrastrar runs truncados.
    if require_both_tasks:
        must_cols = ["circuito1_best_mae","circuito1_final_mae","circuito2_best_mae","circuito2_final_mae"]
        for c in must_cols:
            if c in m.columns:
                m = m[~m[c].isna()]

    print(f"[DEBUG] filtros duros → inicio → {len(df)} runs | post → {len(m)} runs")
    return m

def _comparability_slice(df: pd.DataFrame, strict: bool = True):
    if df.empty:
        return df, {"estrategia":"empty", "kept":0, "dropped":0}
    # Siempre mismo modelo y mismo T
    modes_model = df["model"].dropna().unique().tolist()
    modes_T     = df["T"].dropna().unique().tolist()
    if len(modes_model) > 1:
        top_model = df["model"].value_counts().idxmax()
        df = df[df["model"] == top_model]
    if len(modes_T) > 1:
        top_T = df["T"].value_counts().idxmax()
        df = df[df["T"] == top_T]

    kept_before = len(df)
    if strict:
        # AMP mayoritario (si hay), manteniendo NaN
        if df["amp"].notna().any():
            top_amp = df["amp"].value_counts(dropna=True).idxmax()
            df = df[(df["amp"].isna()) | (df["amp"] == top_amp)]
        # batch_size mayoritario (si hay), manteniendo NaN
        if df["batch_size"].notna().any():
            top_bs = df["batch_size"].value_counts(dropna=True).idxmax()
            df = df[(df["batch_size"].isna()) | (df["batch_size"] == top_bs)]
        strategy = "strict:model+T(+amp+batch)"
    else:
        strategy = "relaxed:model+T"

    kept_after = len(df)
    return df, {"estrategia": strategy, "kept": kept_after, "dropped": kept_before - kept_after}

df_sel0 = _filter_paperset(df_all, require_both_tasks=True)
df_sel, stats = _comparability_slice(df_sel0, strict=STRICT_CFG)
if df_sel.empty and STRICT_CFG:
    print("[WARN] Comparabilidad dejó 0 runs. Relajando AMP y batch_size…")
    df_sel, stats = _comparability_slice(df_sel0, strict=False)

# Copia para informes (se puede filtrar naive aquí si se desea solo de cara a tablas/plots)
df_sel["batch_size_filled"] = df_sel["batch_size"].copy()
df_report = df_sel.copy()
if IGNORE_NAIVE_IN_REPORTS and "method_base" in df_report.columns:
    df_report = df_report[df_report["method_base"] != "naive"]

display(df_sel.head(10))
print(f"[OK] Selección final: {len(df_sel)} runs | estrategia: {stats}")

out_csv = SUMMARY / "selection_table.csv"
df_sel.to_csv(out_csv, index=False)
print("[OK] Selección →", out_csv)

# --- NUEVO: guarda la ruta en _last.json
_last_update(selection=out_csv)


[DEBUG] filtros duros → inicio → 246 runs | post → 174 runs


,run_dir,preset,method,encoder,model,seed,T,batch_size,amp,emissions_kg,elapsed_sec,circuito1_best_mae,circuito1_final_mae,circuito2_best_mae,circuito2_final_mae,circuito1_forget_abs,circuito1_forget_rel,circuito2_forget_abs,circuito2_forget_rel,avg_forget_rel,is_new_runner,mtime,mtime_dt,method_base,batch_size_filled
0,continual_fast_as-snn_gr_0.6_lam_0.36_att_f6_f...,fast,as-snn_gr_0.6_lam_0.36_att_f6,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.005630,771.873337,0.189499,0.262805,0.190483,0.190483,0.073306,0.386843,0.0,0.0,0.193422,True,1.763134e+09,2025-11-14 15:27:35.800247431,as-snn,320.0
1,continual_fast_ewc_lam_5e+07_fast_ewc_l5e7_fb3...,fast,ewc_lam_5e+07,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.008511,1921.457078,0.140196,0.231559,0.199714,0.199714,0.091363,0.651682,0.0,0.0,0.325841,True,1.762962e+09,2025-11-12 15:38:42.516785860,ewc,320.0
2,continual_fast_as-snn_gr_0.6_lam_0.4_att_f6_fa...,fast,as-snn_gr_0.6_lam_0.4_att_f6,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.004914,770.022141,0.127710,0.175707,0.165940,0.165940,0.047996,0.375820,0.0,0.0,0.187910,True,1.763093e+09,2025-11-14 04:09:08.899888992,as-snn,320.0
5,continual_fast_as-snn_gr_0.33_lam_0.9+ewc_lam_...,fast,as-snn_gr_0.33_lam_0.9+ewc_lam_2e+06,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.004565,775.319172,0.039951,0.071067,0.066556,0.066556,0.031116,0.778862,0.0,0.0,0.389431,True,1.762867e+09,2025-11-11 13:12:27.636383533,as-snn,320.0
6,continual_fast_sa-snn_fast3_sa_k0p10_tau24_vt1...,fast,sa-snn,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.005862,807.286098,0.152185,0.238187,0.212210,0.212210,0.086003,0.565120,0.0,0.0,0.282560,True,1.763072e+09,2025-11-13 22:18:04.688407183,sa-snn,320.0
7,continual_fast_sca-snn_bins50_beta0.35_bias0_t...,fast,sca-snn_bins50_beta0.35_bias0_temp0.7_ab16_flat0,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.004905,861.101429,0.138216,0.193343,0.167474,0.167474,0.055127,0.398843,0.0,0.0,0.199422,True,1.762893e+09,2025-11-11 20:28:14.685267210,sca-snn,320.0
8,continual_fast_ewc_lam_1e+09_fast3_ewc_l12e9_f...,fast,ewc_lam_1e+09,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.006649,987.976409,0.142566,0.213006,0.210212,0.210212,0.070440,0.494088,0.0,0.0,0.247044,True,1.763077e+09,2025-11-13 23:37:42.875029564,ewc,320.0
9,continual_fast_rehearsal_buf_5000_rr_20_fast_r...,fast,rehearsal_buf_5000_rr_20,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.006238,1401.138529,0.134941,0.161312,0.168430,0.168430,0.026371,0.195426,0.0,0.0,0.097713,True,1.762937e+09,2025-11-12 08:43:23.035165071,rehearsal,320.0
11,continual_fast_as-snn_gr_0.4_lam_1.5_att_f6_sc...,fast,as-snn_gr_0.4_lam_1.5_att_f6_scale_on,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.004377,741.196114,0.179445,0.255191,0.219784,0.219784,0.075745,0.422109,0.0,0.0,0.211054,True,1.762938e+09,2025-11-12 08:55:47.674229383,as-snn,320.0
12,continual_fast_sca-snn_bins32_beta0.8_bias0.1_...,fast,sca-snn_bins32_beta0.8_bias0.1_temp0.1_ab12_flat1,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.005089,826.357135,0.134410,0.206274,0.167309,0.167309,0.071864,0.534661,0.0,0.0,0.267330,True,1.763103e+09,2025-11-14 06:55:57.203609467,sca-snn,320.0


[OK] Selección final: 155 runs | estrategia: {'estrategia': 'strict:model+T(+amp+batch)', 'kept': 155, 'dropped': 0}
[OK] Selección → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/selection_table.csv
[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


In [6]:
# Celda 5 — Ranking compuesto + winners por método (versión robusta)
def _ensure_mae_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Crea alias entre c2_final_mae y circuito2_final_mae si falta alguno."""
    df = df.copy()
    has_c2    = "c2_final_mae" in df.columns
    has_circ2 = "circuito2_final_mae" in df.columns
    if has_c2 and not has_circ2:
        df["circuito2_final_mae"] = df["c2_final_mae"]
    elif has_circ2 and not has_c2:
        df["c2_final_mae"] = df["circuito2_final_mae"]
    return df

def _minmax_norm(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    if s.size == 0:
        return s
    vals = s.values.astype(float)
    finite = np.isfinite(vals)
    if finite.sum() == 0:
        return pd.Series(np.full(len(s), np.nan), index=s.index)
    lo, hi = float(np.nanmin(vals)), float(np.nanmax(vals))
    if not np.isfinite(lo) or not np.isfinite(hi) or hi <= lo:
        out = np.zeros(len(s), dtype=float)
        out[~finite] = np.nan
        return pd.Series(out, index=s.index)
    return (s - lo) / (hi - lo)

# 1) Copia y compatibilidad de columnas
df_rank = _ensure_mae_columns(df_report)

# Columnas clave
MAE_COL  = "circuito2_final_mae"
FORG_COL = "avg_forget_rel"

# 2) Si está vacío, escribe CSV vacío y sal con aviso
if df_rank.empty:
    winners = df_rank.copy()
    out_csv = SUMMARY / "winners_per_method.csv"
    winners.to_csv(out_csv, index=False)
    print("[WARN] df_rank está vacío tras filtros. Winners vacío →", out_csv)
    display(winners)  # DF vacío
else:
    mae_norm  = _minmax_norm(df_rank[MAE_COL]  if MAE_COL  in df_rank.columns else pd.Series([], dtype=float))
    forg_norm = _minmax_norm(df_rank[FORG_COL] if FORG_COL in df_rank.columns else pd.Series([], dtype=float))

    score_comp = ALPHA_COMPOSITE * mae_norm + (1 - ALPHA_COMPOSITE) * forg_norm
    score_eff  = score_comp.copy()
    if len(score_eff) != 0:
        score_eff[forg_norm.isna()] = mae_norm[forg_norm.isna()]  # fallback

    df_rank["score_comp"] = score_comp
    df_rank["score_eff"]  = score_eff

    # Agrupación: por método base o por método completo (separa composites)
    group_col = "method" if GROUP_BY_FULL_METHOD else "method_base"
    if group_col not in df_rank.columns:
        df_rank[group_col] = df_rank["method"] if "method" in df_rank.columns else "unknown"

    # Winners
    if df_rank["score_eff"].notna().any():
        winners = (
            df_rank
            .sort_values([group_col, "score_eff", MAE_COL], ascending=[True, True, True])
            .groupby(group_col, as_index=False)
            .head(1)
            .sort_values("score_eff", ascending=True)
        )
    else:
        winners = (
            df_rank
            .sort_values([group_col, MAE_COL], ascending=[True, True])
            .groupby(group_col, as_index=False)
            .head(1)
        )

    # Export
    out_csv = SUMMARY / f"winners_per_{'fullmethod' if GROUP_BY_FULL_METHOD else 'methodbase'}.csv"
    winners.to_csv(out_csv, index=False)

    n_rows = len(df_rank)
    n_groups = df_rank[group_col].nunique()
    print(f"[OK] Winners → {out_csv} | candidatos={n_rows} | grupos={n_groups}")
    cols_show = [c for c in ["run_dir", group_col, "seed", MAE_COL, FORG_COL, "emissions_kg", "score_eff"] if c in winners.columns]
    display(winners[cols_show] if cols_show else winners)

# --- NUEVO: guarda la ruta de winners
_last_update(winners=out_csv)
# Mantén "winners" en el entorno por si otras celdas lo usan


[OK] Winners → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/winners_per_methodbase.csv | candidatos=151 | grupos=5


,run_dir,method_base,seed,circuito2_final_mae,avg_forget_rel,emissions_kg,score_eff
175,continual_fast_as-snn_gr_0.5_lam_2.5_att_f6_sm...,as-snn,42.0,0.082089,0.056205,0.004415,0.102395
168,continual_fast_sca-snn_bins50_beta0.55_bias0_t...,sca-snn,42.0,0.043387,0.385824,0.004789,0.114006
132,continual_fast_rehearsal_buf_3000_rr_15_fast_r...,rehearsal,42.0,0.050001,0.361095,0.004529,0.121360
173,continual_fast_sa-snn_fast_sa_k8_tau32_vt1p30_...,sa-snn,42.0,0.045024,0.426407,0.004630,0.129627
222,continual_fast_ewc_lam_3e+06_lam_3e+06_fast_ew...,ewc,42.0,0.064441,0.339687,0.004498,0.147042


[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


In [7]:
# Celda 5.1 — Deltas relativos vs baseline (para narrativa TFM)
def _row_key_for_match(r):
    """Clave de emparejamiento para baseline estricto."""
    return (
        r.get("preset"), r.get("encoder"), r.get("model"), r.get("T"),
        r.get("amp"), r.get("batch_size"), r.get("seed")
    )

def compute_relative_to_baseline(df_in: pd.DataFrame, baseline="naive", strict=True):
    if not baseline:
        return None
    if isinstance(baseline, str) and baseline.lower() == "auto":
        bases = df_in["method_base"].dropna().str.lower().unique().tolist()
        baseline = "ewc" if "ewc" in bases else ("naive" if "naive" in bases else None)
        if not baseline:
            return None
    dfb = df_in.copy()
    dfb["__key__"] = dfb.apply(
        lambda r: (r.get("preset"), r.get("encoder"), r.get("model"), r.get("T"),
                   r.get("amp"), r.get("batch_size"), r.get("seed")), axis=1)

    # Para cada clave, escoge el run baseline con menor MAE y usa sus dos métricas
    base_rows = []
    for key, g in dfb[dfb["method_base"].astype(str).str.lower() == baseline.lower()].groupby("__key__"):
        if g.empty:             continue
        idx = g["circuito2_final_mae"].astype(float).idxmin()
        row = g.loc[idx]
        base_rows.append({
            "__key__": key,
            "baseline_mae": float(row["circuito2_final_mae"]),
            "baseline_forget": float(row["avg_forget_rel"])
        })
    base_tbl = pd.DataFrame(base_rows)

    out = dfb.merge(base_tbl, on="__key__", how="left")
    out["delta_mae_vs_base"]    = out["circuito2_final_mae"] - out["baseline_mae"]
    out["delta_forget_vs_base"] = out["avg_forget_rel"]      - out["baseline_forget"]
    return out

if RELATIVE_BASELINE:
    df_rel = compute_relative_to_baseline(df_sel, baseline=RELATIVE_BASELINE, strict=BASELINE_MATCH_STRICT)
    if df_rel is not None:
        out_csv = SUMMARY / f"relative_to_{RELATIVE_BASELINE}.csv"
        df_rel.to_csv(out_csv, index=False)
        print(f"[OK] Relative-to-baseline → {out_csv}")

        # --- NUEVO: guarda la ruta en _last.json
        _last_update(relative_baseline=RELATIVE_BASELINE, relative_csv=out_csv)

        # Top-10 mejoras vs baseline (orden: delta_mae luego delta_forget)
        mask_has_base = (~df_rel["baseline_mae"].isna()) & (~df_rel["baseline_forget"].isna())
        top_improve = (df_rel[mask_has_base]
                       .sort_values(["delta_mae_vs_base","delta_forget_vs_base"], ascending=[True, True])
                       .head(10))
        cols_show = ["run_dir","method","method_base","circuito2_final_mae","avg_forget_rel",
                     "baseline_mae","baseline_forget","delta_mae_vs_base","delta_forget_vs_base"]
        display(top_improve[cols_show])


[OK] Relative-to-baseline → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/relative_to_ewc.csv
[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


,run_dir,method,method_base,circuito2_final_mae,avg_forget_rel,baseline_mae,baseline_forget,delta_mae_vs_base,delta_forget_vs_base
104,continual_fast_sca-snn_bins50_beta0.55_bias0_t...,sca-snn_bins50_beta0.55_bias0_temp0.3_ab8_flat0,sca-snn,0.043387,0.385824,0.064441,0.339687,-0.021054,0.046136
119,continual_fast_naive_fast_naive_rate_model-Pil...,naive,naive,0.043857,0.075256,0.064441,0.339687,-0.020584,-0.264431
108,continual_fast_sa-snn_fast_sa_k8_tau32_vt1p30_...,sa-snn,sa-snn,0.045024,0.426407,0.064441,0.339687,-0.019417,0.086720
115,continual_fast_sa-snn_smoke_sa_snn_rate_model-...,sa-snn,sa-snn,0.047704,0.465579,0.064441,0.339687,-0.016737,0.125892
10,continual_fast_sca-snn_bins10_beta0.55_bias0_t...,sca-snn_bins10_beta0.55_bias0_temp0.3_ab2_flat0,sca-snn,0.047866,0.448801,0.064441,0.339687,-0.016575,0.109113
23,continual_fast_rehearsal_buf_3000_rr_15_smoke_...,rehearsal_buf_3000_rr_15,rehearsal,0.048710,0.631903,0.064441,0.339687,-0.015731,0.292215
84,continual_fast_rehearsal_buf_3000_rr_15_fast_r...,rehearsal_buf_3000_rr_15,rehearsal,0.050001,0.361095,0.064441,0.339687,-0.014440,0.021407
113,continual_fast_as-snn_gr_0.33_lam_0.9_fast_as_...,as-snn_gr_0.33_lam_0.9,as-snn,0.050001,0.361095,0.064441,0.339687,-0.014440,0.021407
22,continual_fast_as-snn_gr_0.3_lam_1.2_att_f6_fa...,as-snn_gr_0.3_lam_1.2_att_f6,as-snn,0.064408,1.208131,0.064441,0.339687,-0.000032,0.868444
141,continual_fast_ewc_lam_3e+06_lam_3e+06_fast_ew...,ewc_lam_3e+06,ewc,0.064441,0.339687,0.064441,0.339687,0.000000,0.000000


In [8]:
# Celda 5.2 — Porcentajes vs EWC y scorecards por método (sin dependencias extra)
import pandas as pd, numpy as np
from pathlib import Path

# Asegura que tenemos df_rel (de la Celda 5.1)
try:
    df_rel
except NameError:
    df_rel = compute_relative_to_baseline(df_sel, baseline="ewc", strict=True)

# 1) % mejora en MAE y cambio de olvido en puntos porcentuales (pp)
rel = df_rel.copy()
rel["mae_improv_pct"] = 100.0 * (rel["baseline_mae"] - rel["circuito2_final_mae"]) / rel["baseline_mae"]
rel["forget_delta_pp"] = 100.0 * (rel["avg_forget_rel"] - rel["baseline_forget"])

# 2) “Winners” por método ya calculado en Celda 5; si no existe, lo recomputamos rápido
try:
    winners
except NameError:
    tmp = df_rel.copy()
    # Normaliza columnas si faltan
    if "c2_final_mae" not in tmp.columns and "circuito2_final_mae" in tmp.columns:
        tmp["c2_final_mae"] = tmp["circuito2_final_mae"]
    # Score simple si no hay score_eff
    if "score_eff" not in tmp.columns:
        tmp["score_eff"] = np.nan
    group_col = "method_base" if "method_base" in tmp.columns else "method"
    winners = (tmp.sort_values([group_col, "c2_final_mae"], ascending=[True, True])
                   .groupby(group_col, as_index=False).head(1))

# 3) “Scorecard” por método: junta winners con porcentajes vs baseline
key_cols = ["run_dir","method","method_base","preset","encoder","model","seed","T","batch_size","amp"]
cols_metrics = ["circuito2_final_mae","avg_forget_rel","baseline_mae","baseline_forget",
                "mae_improv_pct","forget_delta_pp","emissions_kg","elapsed_sec"]
scorecards = winners[key_cols].merge(rel[key_cols + cols_metrics], on=key_cols, how="left")

# 4) Exporta CSV legible para la memoria
out_dir = SUMMARY
scorecards_csv = out_dir / "scorecards_por_metodo_vs_ewc.csv"
scorecards.to_csv(scorecards_csv, index=False)
print("[OK] Scorecards →", scorecards_csv)

# 5) (Opcional) si tus ángulos están normalizados [-1,1], dar MAE en grados (±25° típico)
STEER_REP = os.environ.get("STEER_REP", "unknown")  # "norm1" si [-1,1], "deg" si ya son grados
DEG_RANGE = float(os.environ.get("DEG_RANGE", "25"))  # cambia si tu rango real es ±X grados

def _mae_to_deg(mae):
    try:
        v = float(mae)
        if STEER_REP.lower() == "norm1":
            return v * DEG_RANGE
        return v  # ya en grados o desconocido
    except Exception:
        return np.nan

scorecards["c2_final_mae_deg"] = scorecards["circuito2_final_mae"].apply(_mae_to_deg)
scorecards.to_csv(scorecards_csv, index=False)  # re-escribe con la columna extra
print("[OK] Añadido c2_final_mae_deg (si procede) →", scorecards_csv)

# 6) Vista rápida ordenada por “mejor” (↑% mejora MAE, ↓Δ olvido)
show_cols = ["method_base","method","circuito2_final_mae","c2_final_mae_deg","avg_forget_rel",
             "baseline_mae","baseline_forget","mae_improv_pct","forget_delta_pp","emissions_kg"]
disp = (scorecards.sort_values(["mae_improv_pct","forget_delta_pp"], ascending=[False, True]))[show_cols]
display(disp)


[OK] Scorecards → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/scorecards_por_metodo_vs_ewc.csv
[OK] Añadido c2_final_mae_deg (si procede) → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/scorecards_por_metodo_vs_ewc.csv


,method_base,method,circuito2_final_mae,c2_final_mae_deg,avg_forget_rel,baseline_mae,baseline_forget,mae_improv_pct,forget_delta_pp,emissions_kg
1,sca-snn,sca-snn_bins50_beta0.55_bias0_temp0.3_ab8_flat0,0.043387,0.043387,0.385824,0.064441,0.339687,32.671692,4.613624,0.004789
3,sa-snn,sa-snn,0.045024,0.045024,0.426407,0.064441,0.339687,30.131477,8.671982,0.004630
2,rehearsal,rehearsal_buf_3000_rr_15,0.050001,0.050001,0.361095,0.064441,0.339687,22.407904,2.140748,0.004529
4,ewc,ewc_lam_3e+06,0.064441,0.064441,0.339687,0.064441,0.339687,0.000000,0.000000,0.004498
0,as-snn,as-snn_gr_0.5_lam_2.5_att_f6,0.082089,0.082089,0.056205,0.064441,0.339687,-27.386647,-28.348214,0.004415


In [9]:
# Celda 6 — Top-N global y Pareto (versión robusta)
# helpers (por si no vienes de la Celda 5)
try:
    _ = _ensure_mae_columns
except NameError:
    def _ensure_mae_columns(df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        has_c2 = "c2_final_mae" in df.columns
        has_circ2 = "circuito2_final_mae" in df.columns
        if has_c2 and not has_circ2:
            df["circuito2_final_mae"] = df["c2_final_mae"]
        elif has_circ2 and not has_c2:
            df["c2_final_mae"] = df["circuito2_final_mae"]
        return df

df_rank = _ensure_mae_columns(df_report if "df_report" in globals() else df_sel.copy())
if "score_eff" not in df_rank.columns:
    df_rank = df_rank.copy()
    df_rank["score_eff"] = np.nan
if "method_base" not in df_rank.columns:
    df_rank["method_base"] = df_rank["method"] if "method" in df_rank.columns else "unknown"

MAE_COL = "circuito2_final_mae" if "circuito2_final_mae" in df_rank.columns else ("c2_final_mae" if "c2_final_mae" in df_rank.columns else "circuito2_final_mae")

# Top-N compuesto
TOPN = 6
if df_rank.empty:
    topn = df_rank.copy()
    print("[WARN] df_rank está vacío; Top-N vacío.")
else:
    cols_sort = [c for c in ["score_eff", MAE_COL] if c in df_rank.columns]
    if not cols_sort:
        topn = df_rank.head(TOPN)
    else:
        topn = df_rank.sort_values(cols_sort, ascending=[True] * len(cols_sort)).head(TOPN)

out_csv = SUMMARY / "top6_composite.csv"
topn.to_csv(out_csv, index=False)
print("[OK] Top-6 →", out_csv)

# --- NUEVO: guarda la ruta
_last_update(top6=out_csv)

cols_show = [c for c in ["run_dir","method_base","seed",MAE_COL,"avg_forget_rel","emissions_kg","score_eff"] if c in topn.columns]
display(topn[cols_show] if cols_show else topn)

# Pareto (minimiza MAE y olvido); NaN -> +inf para no dominar
tmp = _ensure_mae_columns(df_report.copy())
if "method_base" not in tmp.columns:
    tmp["method_base"] = tmp["method"] if "method" in tmp.columns else "unknown"
if MAE_COL not in tmp.columns:
    tmp[MAE_COL] = np.nan

tmp["_olvido_for_pareto"] = pd.to_numeric(tmp.get("avg_forget_rel", np.nan), errors="coerce")
tmp.loc[tmp["_olvido_for_pareto"].isna(), "_olvido_for_pareto"] = np.inf
tmp["_mae_for_pareto"] = pd.to_numeric(tmp.get(MAE_COL, np.nan), errors="coerce")
tmp.loc[tmp["_mae_for_pareto"].isna(), "_mae_for_pareto"] = np.inf

pareto_idx = []
vals = tmp[["_mae_for_pareto", "_olvido_for_pareto"]].values
for i in range(len(tmp)):
    mae_i, forg_i = vals[i]
    dominated = False
    for j in range(len(tmp)):
        if j == i: continue
        mae_j, forg_j = vals[j]
        if (mae_j <= mae_i) and (forg_j <= forg_i) and ((mae_j < mae_i) or (forg_j < forg_i)):
            dominated = True
            break
    if not dominated:
        pareto_idx.append(i)

df_pareto = tmp.iloc[pareto_idx].drop(columns=["_olvido_for_pareto", "_mae_for_pareto"])
out_csv = SUMMARY / "pareto.csv"
df_pareto.to_csv(out_csv, index=False)
print("[OK] Pareto →", out_csv)

# --- NUEVO: guarda la ruta
_last_update(pareto=out_csv)

cols_show = [c for c in ["run_dir","method_base",MAE_COL,"avg_forget_rel"] if c in df_pareto.columns]
display(df_pareto[cols_show] if cols_show else df_pareto)


[OK] Top-6 → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/top6_composite.csv
[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


,run_dir,method_base,seed,circuito2_final_mae,avg_forget_rel,emissions_kg,score_eff
168,continual_fast_sca-snn_bins50_beta0.55_bias0_t...,sca-snn,42.0,0.043387,0.385824,0.004789,NaN
173,continual_fast_sa-snn_fast_sa_k8_tau32_vt1p30_...,sa-snn,42.0,0.045024,0.426407,0.004630,NaN
183,continual_fast_sa-snn_smoke_sa_snn_rate_model-...,sa-snn,42.0,0.047704,0.465579,0.004596,NaN
13,continual_fast_sca-snn_bins10_beta0.55_bias0_t...,sca-snn,42.0,0.047866,0.448801,0.004390,NaN
31,continual_fast_rehearsal_buf_3000_rr_15_smoke_...,rehearsal,42.0,0.048710,0.631903,0.004333,NaN
132,continual_fast_rehearsal_buf_3000_rr_15_fast_r...,rehearsal,42.0,0.050001,0.361095,0.004529,NaN


[OK] Pareto → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/pareto.csv
[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


,run_dir,method_base,circuito2_final_mae,avg_forget_rel
44,continual_fast_ewc_lam_1e+07_lam_1e+07_smoke_e...,ewc,0.070063,0.306996
48,continual_fast_sca-snn_bins32_beta0.8_bias0.12...,sca-snn,0.268958,0.000000
52,continual_fast_rehearsal_buf_3000_rr_33_fast_r...,rehearsal,0.163895,0.056027
78,continual_fast_as-snn_gr_0.5_lam_2.5_att_f6_fa...,as-snn,0.208686,0.008533
132,continual_fast_rehearsal_buf_3000_rr_15_fast_r...,rehearsal,0.050001,0.361095
168,continual_fast_sca-snn_bins50_beta0.55_bias0_t...,sca-snn,0.043387,0.385824
175,continual_fast_as-snn_gr_0.5_lam_2.5_att_f6_sm...,as-snn,0.082089,0.056205
181,continual_fast_as-snn_gr_0.33_lam_0.9_fast_as_...,as-snn,0.050001,0.361095
190,continual_fast_as-snn_gr_0.6_lam_0.35_att_f6_f...,as-snn,0.209745,0.001521
222,continual_fast_ewc_lam_3e+06_lam_3e+06_fast_ew...,ewc,0.064441,0.339687


In [10]:
# Celda 7 — Scatter MAE vs Olvido
import matplotlib.pyplot as plt

def _scatter(df, title, out_png: Path):
    x = pd.to_numeric(df["avg_forget_rel"], errors="coerce")
    y = pd.to_numeric(df["circuito2_final_mae"], errors="coerce")
    plt.figure(figsize=(7,5))
    plt.scatter(x, y)  # sin estilos ni colores específicos
    plt.xlabel("avg_forget_rel (↓ mejor)")
    plt.ylabel("circuito2_final_mae (↓ mejor)")
    plt.title(title)
    plt.grid(True, which="both", linestyle="--", alpha=0.3)
    plt.tight_layout()
    plt.savefig(out_png, dpi=160)
    plt.close()

_scatter(df_sel, "Todos", SUMMARY / "scatter_all.png")
_scatter(winners if "winners" in globals() else df_sel.head(0), "Winners por método", SUMMARY / "scatter_winners.png")
print("[OK] Scatter →", SUMMARY / "scatter_all.png")
print("[OK] Scatter →", SUMMARY / "scatter_winners.png")

# --- NUEVO: guarda las rutas
_last_update(scatter_all=SUMMARY / "scatter_all.png",
             scatter_winners=SUMMARY / "scatter_winners.png")


[OK] Scatter → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/scatter_all.png
[OK] Scatter → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/scatter_winners.png
[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


In [11]:
# Celda 8 — Trazabilidad por run

def explain_sources(run_row):
    rd = _abs_run_dir(run_row["run_dir"])
    exists = {
        "per_task_perf.json": (rd / "per_task_perf.json").exists(),
        "per_task_perf.csv":  (rd / "per_task_perf.csv").exists(),
        "forgetting.json":    (rd / "forgetting.json").exists(),
        "eval_matrix.csv":    (rd / "eval_matrix.csv").exists(),
        "eval_matrix.json":   (rd / "eval_matrix.json").exists(),
        "efficiency_summary.json": (rd / "efficiency_summary.json").exists(),
        "emissions.csv":      (rd / "emissions.csv").exists(),
        "run_row.json":       (rd / "run_row.json").exists(),
        "task_1_circuito1/manifest.json": (rd / "task_1_circuito1/manifest.json").exists(),
    }
    return exists

print("## NaN en selección ##")
cols_check = ["circuito1_best_mae","circuito1_final_mae","circuito2_final_mae","avg_forget_rel"]
mask_nans = df_sel[cols_check].isna().any(axis=1)
df_nans = df_sel[mask_nans].copy()
print(f"[INFO] {len(df_nans)} runs con NaN en {cols_check}:")
display(df_nans[["run_dir","method_base","preset","encoder","seed","T","amp","batch_size"] + cols_check] if len(df_nans) else df_nans)

for _, r in df_sel.iterrows():
    ex = explain_sources(r)
    ok_flags = " | ".join([f"{k} → {'OK' if v else 'NO'}" for k,v in ex.items()])
    print(f"— {r['run_dir']}\n    {ok_flags}")


## NaN en selección ##
[INFO] 0 runs con NaN en ['circuito1_best_mae', 'circuito1_final_mae', 'circuito2_final_mae', 'avg_forget_rel']:


,run_dir,preset,method,encoder,model,seed,T,batch_size,amp,emissions_kg,elapsed_sec,circuito1_best_mae,circuito1_final_mae,circuito2_best_mae,circuito2_final_mae,circuito1_forget_abs,circuito1_forget_rel,circuito2_forget_abs,circuito2_forget_rel,avg_forget_rel,is_new_runner,mtime,mtime_dt,method_base,batch_size_filled


— continual_fast_as-snn_gr_0.6_lam_0.36_att_f6_fast3_as_gr060_lam036_ema090_l1_att_f6_rate_model-PilotNetSNN_66x200_gray_seed_42
    per_task_perf.json → OK | per_task_perf.csv → OK | forgetting.json → OK | eval_matrix.csv → OK | eval_matrix.json → OK | efficiency_summary.json → OK | emissions.csv → OK | run_row.json → OK | task_1_circuito1/manifest.json → OK
— continual_fast_ewc_lam_5e+07_fast_ewc_l5e7_fb300_rate_model-PilotNetSNN_66x200_gray_seed_42
    per_task_perf.json → OK | per_task_perf.csv → OK | forgetting.json → OK | eval_matrix.csv → OK | eval_matrix.json → OK | efficiency_summary.json → OK | emissions.csv → OK | run_row.json → OK | task_1_circuito1/manifest.json → OK
— continual_fast_as-snn_gr_0.6_lam_0.4_att_f6_fast3_as_gr060_lam040_ema095_att_f6_l2_rate_model-PilotNetSNN_66x200_gray_seed_42
    per_task_perf.json → OK | per_task_perf.csv → OK | forgetting.json → OK | eval_matrix.csv → OK | eval_matrix.json → OK | efficiency_summary.json → OK | emissions.csv → OK | run_ro

In [12]:
# Celda 9 — Sanidad naive vs CL

DF = df_sel.copy()
agg = (
    DF.groupby("method_base", dropna=False)
      .agg(forget_median=("avg_forget_rel","median"),
           forget_mean=("avg_forget_rel","mean"),
           mae_mean=("circuito2_final_mae","mean"),
           runs=("run_dir","count"))
      .reset_index()
)
display(agg)

naive_med = agg.loc[agg["method_base"]=="naive","forget_median"].values
cl_med    = agg.loc[agg["method_base"]!="naive","forget_median"].median()
print(f"[CHECK] naive_median_forget={naive_med[0] if len(naive_med) else np.nan} vs CL_median={cl_med}")
if len(naive_med) and (np.isnan(naive_med[0]) or (not np.isnan(cl_med) and naive_med[0] <= cl_med)):
    print("[WARN] Naive NO olvida más que CL. Revisa eval_matrix/forgetting.json o implementación.")



,method_base,forget_median,forget_mean,mae_mean,runs
0,as-snn,0.193422,0.281639,0.172421,33
1,ewc,0.325293,0.336506,0.190883,27
2,naive,0.150602,0.141927,0.140596,4
3,rehearsal,0.136226,0.204383,0.145587,9
4,sa-snn,0.190910,0.223772,0.164183,26
5,sca-snn,0.166903,0.175976,0.167646,56


[CHECK] naive_median_forget=0.15060156417374304 vs CL_median=0.19090976458387732
[WARN] Naive NO olvida más que CL. Revisa eval_matrix/forgetting.json o implementación.


In [13]:
# Celda 9b — Sanity-check EWC (parse logs en carpetas de run)

import glob

def _find_log_files(run_dir: Path):
    pats = ["*.log", "stdout*.txt", "train*.log"]
    files = []
    for p in pats:
        files += list(run_dir.glob(p))
        files += list((run_dir / "task_1_circuito1").glob(p))
        files += list((run_dir / "task_2_circuito2").glob(p))
    return files

def _parse_ewc_lines(text: str):
    # Busca líneas del estilo: [EWC] base=... | pen=... | pen/base=...
    bases, pens, ratios = [], [], []
    for line in text.splitlines():
        if "[EWC]" in line:
            # intenta extraer números
            try:
                # formato flexible
                # ej: [EWC] base=0.007436 | pen=0 | pen/base=0.000
                m_base = re.search(r"base\s*=\s*([0-9.eE+-]+)", line)
                m_pen  = re.search(r"pen\s*=\s*([0-9.eE+-]+)", line)
                m_rat  = re.search(r"pen/base\s*=\s*([0-9.eE+-]+)", line)
                if m_base: bases.append(float(m_base.group(1)))
                if m_pen:  pens.append(float(m_pen.group(1)))
                if m_rat:  ratios.append(float(m_rat.group(1)))
            except Exception:
                pass
    return bases, pens, ratios

rows = []
for _, r in df_sel.iterrows():
    if not str(r.get("method","")).lower().startswith("ewc") and "ewc" not in str(r.get("method","")).lower():
        continue
    rd = _abs_run_dir(r["run_dir"])
    logs = _find_log_files(rd)
    all_bases, all_pens, all_ratios = [], [], []
    for lf in logs:
        try:
            txt = lf.read_text(encoding="utf-8", errors="ignore")
            b, p, q = _parse_ewc_lines(txt)
            all_bases += b; all_pens += p; all_ratios += q
        except Exception:
            pass
    rows.append({
        "run_dir": r["run_dir"],
        "has_logs": len(logs) > 0,
        "pen_gt0_count": int(sum(1 for x in all_pens if (isinstance(x,float) and x > 0))),
        "pen_entries": len(all_pens),
        "ratio_gt0_count": int(sum(1 for x in all_ratios if (isinstance(x,float) and x > 0))),
        "ratio_entries": len(all_ratios),
    })

df_ewc_sanity = pd.DataFrame(rows)
out_csv = SUMMARY / "ewc_sanity.csv"
df_ewc_sanity.to_csv(out_csv, index=False)
print("[OK] EWC sanity →", out_csv)
display(df_ewc_sanity)


[OK] EWC sanity → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/ewc_sanity.csv


,run_dir,has_logs,pen_gt0_count,pen_entries,ratio_gt0_count,ratio_entries
0,continual_fast_ewc_lam_5e+07_fast_ewc_l5e7_fb3...,False,0,0,0,0
1,continual_fast_as-snn_gr_0.33_lam_0.9+ewc_lam_...,False,0,0,0,0
2,continual_fast_ewc_lam_1e+09_fast3_ewc_l12e9_f...,False,0,0,0,0
3,continual_fast_ewc_lam_6e+06_fast_ewc_l6e6_fb3...,False,0,0,0,0
4,continual_fast_ewc_lam_6e+08_fast3_ewc_l60e8_f...,False,0,0,0,0
5,continual_fast_ewc_lam_4e+08_fast3_ewc_l35e8_f...,False,0,0,0,0
6,continual_fast_ewc_lam_5e+08_fast3_ewc_l50e8_f...,False,0,0,0,0
7,continual_fast_ewc_lam_3e+08_fast_ewc_l3e8_fb3...,False,0,0,0,0
8,continual_fast_ewc_lam_3e+08_fast3_ewc_l30e8_f...,False,0,0,0,0
9,continual_fast_ewc_lam_1e+07_lam_1e+07_smoke_e...,False,0,0,0,0


In [14]:
# Celda 10 — Export final
print("[OK] Artifacts en:", SUMMARY)
for p in sorted(SUMMARY.glob("*.csv")) + sorted(SUMMARY.glob("*.png")):
    print(" -", p.name)

# --- NUEVO: apunta dónde quedó el manifest de “último”
print("[LAST] Manifiesto final:", OUT / "summary" / "_last.json")


[OK] Artifacts en: /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427
 - ewc_sanity.csv
 - pareto.csv
 - relative_to_ewc.csv
 - results_table_fromdisk.csv
 - scorecards_por_metodo_vs_ewc.csv
 - selection_table.csv
 - top6_composite.csv
 - winners_per_methodbase.csv
 - scatter_all.png
 - scatter_winners.png
[LAST] Manifiesto final: /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


In [15]:
# Celda X — Top-5 por método + parámetros (lee rutas desde _last.json y exporta en SUMMARY)

from pathlib import Path
import os, json, re
import pandas as pd
import numpy as np

# --- Fallbacks por si no vienes de la Celda 0
try:
    OUT
except NameError:
    OUT = Path(os.environ.get("OUT_DIR", "/home/cesar/proyectos/TFM_SNN/outputs")).resolve()

try:
    _last_update
except NameError:
    def _last_update(**kwargs):  # no-op si no está definida
        pass

LAST_PATH = OUT / "summary" / "_last.json"
LATEST = OUT / "summary" / "latest"

def _pick_latest_selection():
    # 1) _last.json
    try:
        last = json.loads(LAST_PATH.read_text(encoding="utf-8"))
        p = Path(last.get("selection", ""))
        if p.exists():
            return p
    except Exception:
        pass
    # 2) SUMMARY actual
    try:
        if "SUMMARY" in globals():
            p = Path(SUMMARY) / "selection_table.csv"
            if p.exists():
                return p
    except Exception:
        pass
    # 3) symlink latest
    try:
        p = LATEST / "selection_table.csv"
        if p.exists():
            return p
    except Exception:
        pass
    # 4) glob: el más reciente
    cands = list(OUT.glob("summary/*/selection_table.csv"))
    if not cands:
        raise FileNotFoundError("No se encontró ningún selection_table.csv en outputs/summary/**/")
    cands.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    return cands[0]

# 1) Carga selección
sel_csv = _pick_latest_selection()
df = pd.read_csv(sel_csv)
print(f"[LOAD] selection_table.csv → {sel_csv}")

# 2) Normaliza columnas clave
def _canonical_method(m: str) -> str:
    s = (m or "").lower()
    if s.startswith("ewc"): return "ewc"
    if s.startswith("as-snn"): return "as-snn"
    if s.startswith("sa-snn"): return "sa-snn"
    if s.startswith("sca-snn"): return "sca-snn"
    if s.startswith("rehearsal"): return "rehearsal"
    if s.startswith("naive"): return "naive"
    return s or "unknown"

if "method_base" not in df.columns or df["method_base"].isna().all():
    df["method_base"] = df["method"].astype(str).apply(_canonical_method)
else:
    # rellena NA con prefijo del method o con canonical
    mask_na = df["method_base"].isna()
    if mask_na.any():
        df.loc[mask_na, "method_base"] = (
            df.loc[mask_na, "method"].astype(str).apply(_canonical_method)
        )

# 3) Parser de hiperparámetros desde 'method' (robusto a variaciones de nombres)
_float = lambda s: (None if s is None else float(str(s).replace("_", "").replace("p", ".")))

def parse_params(m):
    s = str(m or "").lower()
    out = {}

    # -------- AS-SNN --------
    # as-snn_gr_<float>_lam_<float>_att_f<d>   + flags: scale_on, ema\d+, _l1/_l2
    m_as = re.search(r"as-snn(?:(?:_gr[_-]?([\d._e+-]+)))?(?:_lam[_-]?([\d._e+-]+))?(?:_att[_-]?f(\d))?", s)
    if m_as:
        gr, lam, att = m_as.groups()
        if gr:  out["gr"] = _float(gr)
        if lam: out["lam"] = lam  # mantenemos texto por si es 3e+08
        if att: out["attach"] = f"f{att}"
    if "scale_on" in s: out["scale_on"] = 1
    m_ema = re.search(r"ema0?(\d+)", s)
    if m_ema: out["ema"] = int(m_ema.group(1))
    if "_l1" in s: out["penalty"] = ("l1" if "penalty" not in out else out["penalty"] + "/l1")
    if "_l2" in s: out["penalty"] = ("l2" if "penalty" not in out else out["penalty"] + "/l2")

    # -------- SA-SNN --------
    # sa-snn_k<d>_tau<d>_vt<1p33|1.33>_p<nnn|2m> + reset1 + flat[01]
    m_sa = re.search(r"sa(?:-snn)?(?:.*?_k(\d+))?(?:_tau(\d+))?(?:_vt([0-9p\._+-]+))?(?:_p([\d_]+|(\d+)m))?", s)
    if m_sa:
        k, tau, vt, p_raw, p_m = m_sa.groups()
        if k:   out["k"] = int(k)
        if tau: out["tau"] = int(tau)
        if vt:  out["vt"] = float(str(vt).replace("_","").replace("p","."))
        if p_m: out["p"] = int(p_m) * 1_000_000
        elif p_raw:
            out["p"] = int(str(p_raw).replace("_",""))
    if "reset1" in s: out["reset"] = 1
    if "flat1"  in s: out["flat"]  = 1
    elif "flat0" in s: out["flat"] = 0

    # -------- SCA-SNN --------
    # sca-snn_bins<d>_beta<flt>_bias<flt>_temp<flt>_ab<d+>
    m_sca = re.search(r"sca(?:-snn)?_bins(\d+).*?beta([-\d\.e+]+).*?bias([-\d\.e+]+).*?temp([-\d\.e+]+).*?(ab\d+)", s)
    if m_sca:
        bins, beta, bias, temp, ab = m_sca.groups()
        out.update({
            "bins": int(bins),
            "beta": float(beta),
            "bias": float(bias),
            "temp": float(temp),
            "attach_block": ab
        })

    # -------- Rehearsal --------
    # rehearsal_buf_<int>_rr_<0-100 | 0.x>
    m_reh = re.search(r"rehearsal.*?_buf[_-]?(\d+).*?_rr[_-]?([0-9\.]+)", s)
    if m_reh:
        buf, rr = m_reh.groups()
        rr_f = float(rr)
        rr_f = rr_f/100.0 if rr_f > 1.0 else rr_f
        out.update({"buffer": int(buf), "rr": rr_f})

    # -------- EWC --------
    # ewc_*_lam_<3e+08>  + fisher batches: fb\d+ o 'f\d' (evita confundir con attach)
    m_ewc = re.search(r"\bewc\b.*?_lam[_-]?([\de\+\-\.]+)", s)
    if m_ewc:
        out["lambda"] = m_ewc.group(1)
    m_fb = re.search(r"\bfb(\d+)\b", s) or re.search(r"\bf(\d+)\b", s)
    if "ewc" in s and m_fb:
        out["fisher_batches"] = int(m_fb.group(1))

    # -------- genérico: attach fX si aparece suelto y no viene de AS-SNN
    if "attach" not in out:
        m_att = re.search(r"\bf(\d)\b", s)
        if m_att:
            out["attach"] = f"f{m_att.group(1)}"

    return out

params = df["method"].apply(parse_params).apply(pd.Series)
dfp = pd.concat([df, params], axis=1)

# 4) Métrica objetivo y orden
for col_src, col_dst in [("circuito2_final_mae","c2_final_mae"),
                         ("avg_forget_rel","forget"),
                         ("emissions_kg","co2")]:
    if col_src in dfp.columns:
        dfp[col_dst] = pd.to_numeric(dfp[col_src], errors="coerce")
    else:
        dfp[col_dst] = np.nan

# 5) Top-5 por method_base
tops = []
for mb, g in dfp.groupby("method_base", dropna=False):
    g = g.sort_values(["c2_final_mae","forget","co2"], ascending=[True, True, True]).head(5)
    # columnas de hiperparámetros presentes dinámicamente
    param_cols = [c for c in [
        "gr","lam","attach","k","tau","vt","p","bins","beta","bias","temp",
        "attach_block","flat","buffer","rr","lambda","fisher_batches","scale_on","ema","penalty","reset"
    ] if c in g.columns]
    keep_cols = ["method_base","method","c2_final_mae","forget","co2"] + param_cols
    tops.append(g[keep_cols])

if tops:
    top5 = pd.concat(tops, ignore_index=True)
else:
    top5 = dfp.head(0)

# 6) Exporta en el SUMMARY correspondiente a la selección
export_dir = sel_csv.parent   # mismo SUMMARY donde está selection_table.csv
out_csv = export_dir / "top5_por_metodo.csv"
out_tex = export_dir / "top5_por_metodo.tex"

top5.to_csv(out_csv, index=False)

with open(out_tex, "w", encoding="utf-8") as f:
    for mb, g in top5.groupby("method_base", dropna=False):
        # ordena columnas: primero métricas, luego params
        base_cols = ["method","c2_final_mae","forget","co2"]
        other_cols = [c for c in g.columns if c not in (["method_base"] + base_cols)]
        gg = g[base_cols + other_cols]
        f.write(gg.to_latex(index=False, float_format="%.6f"))

print(f"[OK] → {out_csv.name} / {out_tex.name} en {export_dir}")

# registra rutas para reutilización
_last_update(top5_csv=out_csv, top5_tex=out_tex)


[LOAD] selection_table.csv → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/selection_table.csv
[OK] → top5_por_metodo.csv / top5_por_metodo.tex en /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427
[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


In [16]:
# Celda A — Cargar y mostrar el Top-5 por método (y ruta de dónde salió)

from pathlib import Path
import json, os
import pandas as pd

OUT = Path(os.environ.get("OUT_DIR", "/home/cesar/proyectos/TFM_SNN/outputs")).resolve()
LAST = OUT / "summary" / "_last.json"

def _pick_top5_path():
    # 1) _last.json si existe
    try:
        j = json.loads(LAST.read_text(encoding="utf-8"))
        p = Path(j.get("top5_csv",""))
        if p.exists(): 
            return p
        # fallback a la carpeta del selection si solo tenemos eso
        p2 = Path(j.get("selection","")).parent / "top5_por_metodo.csv"
        if p2.exists():
            return p2
    except Exception:
        pass
    # 2) symlink 'latest'
    p = OUT / "summary" / "latest" / "top5_por_metodo.csv"
    if p.exists(): 
        return p
    # 3) glob más reciente
    cands = sorted(OUT.glob("summary/*/top5_por_metodo.csv"), key=lambda x: x.stat().st_mtime, reverse=True)
    if not cands:
        raise FileNotFoundError("No encuentro top5_por_metodo.csv; ejecuta antes la celda de Top-5.")
    return cands[0]

top5_csv = _pick_top5_path()
top5 = pd.read_csv(top5_csv)
print(f"[OK] Cargado Top-5 → {top5_csv}")
display(top5)

# Vista separada por método (para inspección rápida en el notebook)
for mb, g in top5.groupby("method_base", dropna=False):
    print(f"\n### {mb} — {len(g)} configs")
    display(g)


[OK] Cargado Top-5 → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/top5_por_metodo.csv


,method_base,method,c2_final_mae,forget,co2,gr,bins,beta,bias,temp,attach_block,flat,buffer,rr,scale_on,penalty
0,as-snn,as-snn_gr_0.33_lam_0.9,0.050001,0.361095,0.004369,0.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,as-snn,as-snn_gr_0.3_lam_1.2_att_f6,0.064408,1.208131,0.004567,0.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,as-snn,as-snn_gr_0.33_lam_0.9+ewc_lam_2e+06,0.066556,0.389431,0.004565,0.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,as-snn,as-snn_gr_0.5_lam_2.5_att_f6,0.082089,0.056205,0.004415,0.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,as-snn,as-snn_gr_0.3_lam_1.2_att_f6_scale_on,0.089654,1.692113,0.004462,0.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
5,ewc,ewc_lam_3e+06,0.064441,0.339687,0.004498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ewc,ewc_lam_1e+07,0.070063,0.306996,0.005054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ewc,ewc_lam_1e+09,0.085984,0.586833,0.004318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ewc,ewc_lam_6e+07,0.186387,0.257757,0.007527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ewc,ewc_lam_2e+08,0.189092,0.415879,0.006648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



### as-snn — 5 configs


,method_base,method,c2_final_mae,forget,co2,gr,bins,beta,bias,temp,attach_block,flat,buffer,rr,scale_on,penalty
0,as-snn,as-snn_gr_0.33_lam_0.9,0.050001,0.361095,0.004369,0.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,as-snn,as-snn_gr_0.3_lam_1.2_att_f6,0.064408,1.208131,0.004567,0.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,as-snn,as-snn_gr_0.33_lam_0.9+ewc_lam_2e+06,0.066556,0.389431,0.004565,0.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,as-snn,as-snn_gr_0.5_lam_2.5_att_f6,0.082089,0.056205,0.004415,0.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,as-snn,as-snn_gr_0.3_lam_1.2_att_f6_scale_on,0.089654,1.692113,0.004462,0.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN



### ewc — 5 configs


...

,method_base,method,c2_final_mae,forget,co2,gr,bins,beta,bias,temp,attach_block,flat,buffer,rr,scale_on,penalty
14,rehearsal,rehearsal_buf_3000_rr_15,0.048710,0.631903,0.004333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,0.15,NaN,NaN
15,rehearsal,rehearsal_buf_3000_rr_15,0.050001,0.361095,0.004529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,0.15,NaN,NaN
16,rehearsal,rehearsal_buf_3000_rr_25,0.162982,0.078468,0.005695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,0.25,NaN,NaN
17,rehearsal,rehearsal_buf_3000_rr_33,0.163895,0.056027,0.004954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,0.33,NaN,NaN
18,rehearsal,rehearsal_buf_5000_rr_20,0.168430,0.097713,0.006238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5000.0,0.20,NaN,NaN



### sa-snn — 5 configs


,method_base,method,c2_final_mae,forget,co2,gr,bins,beta,bias,temp,attach_block,flat,buffer,rr,scale_on,penalty
19,sa-snn,sa-snn,0.045024,0.426407,0.004630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,sa-snn,sa-snn,0.047704,0.465579,0.004596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,sa-snn,sa-snn,0.156119,0.062713,0.004412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,sa-snn,sa-snn,0.156531,0.167736,0.005281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,sa-snn,sa-snn,0.157522,0.157920,0.005827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



### sca-snn — 5 configs


,method_base,method,c2_final_mae,forget,co2,gr,bins,beta,bias,temp,attach_block,flat,buffer,rr,scale_on,penalty
24,sca-snn,sca-snn_bins50_beta0.55_bias0_temp0.3_ab8_flat0,0.043387,0.385824,0.004789,NaN,50.0,0.55,0.00,0.30,ab8,0.0,NaN,NaN,NaN,NaN
25,sca-snn,sca-snn_bins10_beta0.55_bias0_temp0.3_ab2_flat0,0.047866,0.448801,0.004390,NaN,10.0,0.55,0.00,0.30,ab2,0.0,NaN,NaN,NaN,NaN
26,sca-snn,sca-snn_bins50_beta0.52_bias0_temp0.7_ab24_fla...,0.158612,0.098270,0.004476,NaN,50.0,0.52,0.00,0.70,ab24,0.0,NaN,NaN,NaN,NaN
27,sca-snn,sca-snn_bins50_beta0.5_bias0_temp0.45_ab32_flat1,0.158943,0.087051,0.004562,NaN,50.0,0.50,0.00,0.45,ab32,1.0,NaN,NaN,NaN,NaN
28,sca-snn,sca-snn_bins32_beta0.8_bias0.08_temp0.12_ab12_...,0.160026,0.120142,0.004915,NaN,32.0,0.80,0.08,0.12,ab12,1.0,NaN,NaN,NaN,NaN


In [17]:
# Celda B — Una tabla .tex por método + bundle con \input{...}

from pathlib import Path
import pandas as pd
import os, json

OUT = Path(os.environ.get("OUT_DIR", "/home/cesar/proyectos/TFM_SNN/outputs")).resolve()
LAST = OUT / "summary" / "_last.json"

def _load_top5():
    j = {}
    try:
        j = json.loads(LAST.read_text(encoding="utf-8"))
    except Exception:
        pass
    # ruta principal
    p = Path(j.get("top5_csv","")) if j.get("top5_csv") else OUT / "summary" / "latest" / "top5_por_metodo.csv"
    if not p.exists():
        # último disponible
        cands = sorted(OUT.glob("summary/*/top5_por_metodo.csv"), key=lambda x: x.stat().st_mtime, reverse=True)
        if not cands:
            raise FileNotFoundError("No encuentro top5_por_metodo.csv; ejecuta antes la celda de Top-5.")
        p = cands[0]
    df = pd.read_csv(p)
    return df, p.parent

top5, SUMMARY_DIR = _load_top5()
TABLES_DIR = SUMMARY_DIR / "tables_by_method"
TABLES_DIR.mkdir(exist_ok=True, parents=True)

bundle_lines = []
for mb, g in top5.groupby("method_base", dropna=False):
    mb_slug = str(mb or "unknown").replace("/", "-")
    # ordena columnas: métricas al frente
    base_cols = [c for c in ["method","c2_final_mae","forget","co2"] if c in g.columns]
    other_cols = [c for c in g.columns if c not in (["method_base"] + base_cols)]
    gg = g[base_cols + other_cols]

    tex_path = TABLES_DIR / f"top5_{mb_slug}.tex"
    caption = f"Top-5 configuraciones para {mb}"
    label = f"tab:top5_{mb_slug}"
    with open(tex_path, "w", encoding="utf-8") as f:
        f.write("\\begin{table}[ht]\n\\centering\n")
        f.write(gg.to_latex(index=False, float_format="%.6f", escape=True))
        # 👇 evita f-string en \end{table} para no interpolar {table}
        f.write(f"\\caption{{{caption}}}\n\\label{{{label}}}\n" + "\\end{table}\n")

    bundle_lines.append(f"\\input{{{tex_path.as_posix()}}}")

bundle_path = SUMMARY_DIR / "top5_methods_bundle.tex"
bundle_path.write_text("\n\n".join(bundle_lines), encoding="utf-8")

print("[OK] Tablas por método en:", TABLES_DIR)
print("[OK] Bundle para la memoria →", bundle_path)


[OK] Tablas por método en: /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/tables_by_method
[OK] Bundle para la memoria → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/top5_methods_bundle.tex


In [18]:
# Celda C — Genera interpretación en Markdown y LaTeX por método (tendencias de los Top-5)

from pathlib import Path
import pandas as pd, numpy as np, json, os
from scipy.stats import spearmanr

OUT = Path(os.environ.get("OUT_DIR", "/home/cesar/proyectos/TFM_SNN/outputs")).resolve()
LAST = OUT / "summary" / "_last.json"

PARAM_MEANINGS = {
    "as-snn": {
        "lam": "Peso de la regularización/escala sináptica (λ).",
        "gr": "Factor de crecimiento/ajuste de escala sináptica.",
        "attach": "Bloque/feature donde se inserta el módulo (f4/f5/f6...).",
        "scale_on": "Escalado sináptico activado (1) o no (0).",
        "ema": "Suavizado EMA de estadísticas internas.",
        "penalty": "Regularización adicional (L1/L2).",
    },
    "sa-snn": {
        "k": "Número de unidades (top-k) que se mantienen activas por bloque (selectividad).",
        "tau": "Constante temporal del LIF/ventana de actividad.",
        "vt": "Escala/umbral de disparo efectivo.",
        "p": "Presupuesto de activaciones/contadores (cap de actividad).",
        "reset": "Reinicia contadores por tarea (1) o no (0).",
        "flat": "Aplana espacialmente antes de la selección (1/0).",
    },
    "sca-snn": {
        "bins": "Nº de bins para el agrupado/contexto.",
        "beta": "Peso de similitud/contexto.",
        "bias": "Sesgo aplicado en reasignación/expansión.",
        "temp": "Temperatura (suavizado) en selección.",
        "attach_block": "Bloque donde se acopla el módulo (abX).",
    },
    "rehearsal": {
        "buffer": "Tamaño del buffer de ejemplos guardados.",
        "rr": "Ratio de re-revisión (0–1).",
    },
    "ewc": {
        "lambda": "Fuerza EWC (λ) para preservar parámetros importantes.",
        "fisher_batches": "Nº de batches usados para estimar la matriz de Fisher.",
    },
}

def _load_top5():
    j = {}
    try:
        j = json.loads(LAST.read_text(encoding="utf-8"))
    except Exception:
        pass
    p = Path(j.get("top5_csv","")) if j.get("top5_csv") else OUT / "summary" / "latest" / "top5_por_metodo.csv"
    if not p.exists():
        cands = sorted(OUT.glob("summary/*/top5_por_metodo.csv"), key=lambda x: x.stat().st_mtime, reverse=True)
        if not cands:
            raise FileNotFoundError("No top5_por_metodo.csv")
        p = cands[0]
    return pd.read_csv(p), p.parent

def _is_num(s):
    try:
        float(s)
        return True
    except Exception:
        return False

top5, SUMMARY_DIR = _load_top5()
md_lines, tex_lines = [], []

for mb, g in top5.groupby("method_base", dropna=False):
    mb = str(mb or "unknown")
    meanings = PARAM_MEANINGS.get(mb.lower(), {})
    # columnas param candidatas = todo salvo identificadores/metrics
    id_cols = {"method_base","method","c2_final_mae","forget","co2"}
    param_cols = [c for c in g.columns if c not in id_cols and g[c].notna().any()]
    # escoge solo numéricas para correlación
    num_cols = [c for c in param_cols if g[c].apply(_is_num).all()]

    # estadísticos
    stats = []
    for c in num_cols:
        s = pd.to_numeric(g[c], errors="coerce")
        stats.append((c, float(np.nanmedian(s)), float(np.nanmin(s)), float(np.nanmax(s))))
    # correlaciones (Spearman) con métricas
    cors = []
    for c in num_cols:
        x = pd.to_numeric(g[c], errors="coerce")
        y_mae = pd.to_numeric(g["c2_final_mae"], errors="coerce")
        y_for = pd.to_numeric(g["forget"], errors="coerce")
        if x.nunique(dropna=True) >= 3:
            try:
                r_mae, _ = spearmanr(x, y_mae, nan_policy="omit")
                r_for, _ = spearmanr(x, y_for, nan_policy="omit")
                cors.append((c, float(r_mae), float(r_for)))
            except Exception:
                pass

    # --- Markdown ---
    md_lines.append(f"### {mb}")
    md_lines.append("**Parámetros y significado**:")
    for c, desc in meanings.items():
        if c in g.columns:
            md_lines.append(f"- `{c}`: {desc}")
    if stats:
        md_lines.append("**Tendencias en el Top-5 (mediana [mín–máx])**:")
        for c, med, lo, hi in stats:
            md_lines.append(f"- `{c}` ≈ {med:.4g} [{lo:.4g}–{hi:.4g}]")
    if cors:
        md_lines.append("**Correlación (Spearman) con métricas**:")
        for c, r1, r2 in cors:
            md_lines.append(f"- `{c}` vs MAE: {r1:+.2f} | vs olvido: {r2:+.2f}")
    md_lines.append("")

    # --- LaTeX ---
    tex_lines.append(f"\\subsubsection*{{{mb}}}")
    if meanings:
        tex_lines.append("\\textbf{Parámetros y significado}:\\par")
        tex_lines.append("\\begin{itemize}")
        for c, desc in meanings.items():
            if c in g.columns:
                tex_lines.append(f"\\item \\texttt{{{c}}}: {desc}")
        tex_lines.append("\\end{itemize}")
    if stats:
        tex_lines.append("\\textbf{Tendencias en el Top-5 (mediana [mín–máx])}:\\par")
        tex_lines.append("\\begin{itemize}")
        for c, med, lo, hi in stats:
            tex_lines.append(f"\\item \\texttt{{{c}}} $\\approx$ {med:.4g} [{lo:.4g}–{hi:.4g}]")
        tex_lines.append("\\end{itemize}")
    if cors:
        tex_lines.append("\\textbf{Correlación (Spearman) con métricas}:\\par")
        tex_lines.append("\\begin{itemize}")
        for c, r1, r2 in cors:
            tex_lines.append(f"\\item \\texttt{{{c}}} vs MAE: {r1:+.2f} \\;|\\; vs olvido: {r2:+.2f}")
        tex_lines.append("\\end{itemize}")

MD_PATH  = SUMMARY_DIR / "interpretacion_top5_methods.md"
TEX_PATH = SUMMARY_DIR / "interpretacion_top5_methods.tex"
MD_PATH.write_text("\n".join(md_lines), encoding="utf-8")
TEX_PATH.write_text("\n".join(tex_lines), encoding="utf-8")
print("[OK] Interpretación Markdown →", MD_PATH)
print("[OK] Interpretación LaTeX   →", TEX_PATH)


[OK] Interpretación Markdown → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/interpretacion_top5_methods.md
[OK] Interpretación LaTeX   → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/interpretacion_top5_methods.tex


In [19]:
# Celda C — Tablas y resumen por método en Word (sin SciPy, sin LaTeX)
from pathlib import Path
import pandas as pd, numpy as np, json, os, math
from datetime import datetime

# ====== Paths y carga del Top-5 detectado automáticamente ======
OUT = Path(os.environ.get("OUT_DIR", "/home/cesar/proyectos/TFM_SNN/outputs")).resolve()
LAST = OUT / "summary" / "_last.json"

def _load_top5():
    # Intenta leer el puntero a la última ejecución
    j = {}
    try:
        j = json.loads(LAST.read_text(encoding="utf-8"))
    except Exception:
        pass
    p = Path(j.get("top5_csv","")) if j.get("top5_csv") else OUT / "summary" / "latest" / "top5_por_metodo.csv"
    if not p.exists():
        cands = sorted(OUT.glob("summary/*/top5_por_metodo.csv"),
                       key=lambda x: x.stat().st_mtime, reverse=True)
        if not cands:
            raise FileNotFoundError("No existe top5_por_metodo.csv; ejecuta antes la celda que lo genera.")
        p = cands[0]
    return pd.read_csv(p), p.parent

top5, SUMMARY_DIR = _load_top5()

# ====== Diccionario de significados (se muestra solo si la columna existe) ======
PARAM_MEANINGS = {
    "as-snn": {
        "lam": "Peso de la regularización/escala sináptica (λ).",
        "gr": "Factor de crecimiento/ajuste de escala sináptica.",
        "attach": "Bloque/feature donde se inserta el módulo (f4/f5/f6...).",
        "scale_on": "Escalado sináptico activado (1) o no (0).",
        "ema": "Suavizado EMA de estadísticas internas.",
        "penalty": "Regularización adicional (L1/L2).",
    },
    "sa-snn": {
        "k": "Número de unidades activas (top-k) por bloque.",
        "tau": "Constante temporal del LIF/ventana de actividad.",
        "vt": "Escala/umbral de disparo efectivo.",
        "p": "Presupuesto de activaciones/contadores.",
        "reset": "Reinicia contadores por tarea (1/0).",
        "flat": "Aplana espacialmente antes de la selección (1/0).",
    },
    "sca-snn": {
        "bins": "Número de bins para el agrupado/contexto.",
        "beta": "Peso de similitud/contexto.",
        "bias": "Sesgo en reasignación/expansión.",
        "temp": "Temperatura (suavizado) en selección.",
        "attach_block": "Bloque donde se acopla el módulo (abX).",
    },
    "rehearsal": {
        "buffer": "Tamaño del buffer de ejemplos.",
        "rr": "Ratio de re-revisión (0–1).",
    },
    "ewc": {
        "lambda": "Fuerza EWC (λ) para preservar parámetros importantes.",
        "fisher_batches": "Número de batches para estimar Fisher.",
    },
}
# Alias de nombres (por si el parser llamó 'pM' a 'p', etc.)
PARAM_ALIASES = {
    "sa-snn": {"p": ["p", "pM"]},
}

# ====== Utilidades ======
def _is_num(s):
    try:
        float(s); return True
    except Exception:
        return False

def _spearman_safe(a, b) -> float:
    """Spearman sin SciPy: rangos + correlación de Pearson entre rangos."""
    ax = pd.to_numeric(pd.Series(a), errors="coerce")
    bx = pd.to_numeric(pd.Series(b), errors="coerce")
    mask = ax.notna() & bx.notna()
    ax = ax[mask]; bx = bx[mask]
    if len(ax) < 3:  # sin variación suficiente
        return float("nan")
    ra = ax.rank(method="average")
    rb = bx.rank(method="average")
    if ra.nunique() < 2 or rb.nunique() < 2:
        return float("nan")
    r = np.corrcoef(ra.to_numpy(), rb.to_numpy())[0, 1]
    return float(r)

def _format_float(x, nd=6):
    try:
        v = float(x)
        if math.isnan(v): return ""
        return f"{v:.{nd}f}"
    except Exception:
        return str(x)

# ====== Preparación de columnas y orden ======
# Identificadores y métricas "core"
ID_COLS = ["method_base","method"]
METRIC_COLS = ["c2_final_mae","forget","co2"]  # salieron así del Top-5
for c in METRIC_COLS:
    if c in top5.columns:
        top5[c] = pd.to_numeric(top5[c], errors="coerce")

# Columnas de parámetros = el resto que no sean ID/Metrics
PARAM_COLS_ALL = [c for c in top5.columns if c not in set(ID_COLS + METRIC_COLS)]

# Ordena cada grupo por métricas (menor MAE, menor olvido, menor CO₂)
def _sort_group(g: pd.DataFrame):
    cols = [c for c in METRIC_COLS if c in g.columns]
    if cols:
        return g.sort_values(cols, ascending=[True]*len(cols))
    return g

# ====== Generación del documento Word ======
docx_path = SUMMARY_DIR / "top5_por_metodo.docx"

try:
    from docx import Document
    from docx.shared import Pt, Inches
    from docx.enum.text import WD_ALIGN_PARAGRAPH

    doc = Document()
    # Portada sencilla
    title = doc.add_heading("Top-5 por método (Tablas y Resumen)", level=1)
    title.alignment = WD_ALIGN_PARAGRAPH.LEFT
    meta_p = doc.add_paragraph(
        f"Generado: {datetime.now().strftime('%Y-%m-%d %H:%M')}  ·  Carpeta: {SUMMARY_DIR.name}"
    )
    meta_p.style.font.size = Pt(9)

    # Una sección por método_base
    for mb, g in top5.groupby("method_base", dropna=False):
        mb = str(mb or "unknown")
        g = _sort_group(g).reset_index(drop=True)
        doc.add_heading(mb, level=2)

        # 1) Parámetros y significado (solo los presentes)
        meanings = PARAM_MEANINGS.get(mb.lower(), {})
        aliases  = PARAM_ALIASES.get(mb.lower(), {})
        bullets = []
        for param_key, desc in meanings.items():
            candidates = [param_key] + aliases.get(param_key, [])
            present = next((cc for cc in candidates if cc in g.columns and g[cc].notna().any()), None)
            if present:
                bullets.append((present, desc))
        if bullets:
            doc.add_paragraph("Parámetros y significado:", style=None)
            for name, desc in bullets:
                par = doc.add_paragraph(style=None)
                run = par.add_run(f"• {name}: {desc}")

        # 2) Tabla Top-5 con métricas y parámetros detectados
        #    (para no desbordar, priorizamos columnas con más no-NaN)
        param_cols_present = [c for c in PARAM_COLS_ALL if c in g.columns and g[c].notna().any()]
        param_cols_present = sorted(
            param_cols_present,
            key=lambda c: g[c].notna().sum(),
            reverse=True
        )
        # Puedes ajustar este máximo si la tabla queda muy ancha:
        MAX_PARAM_COLS = 12
        param_cols_table = param_cols_present[:MAX_PARAM_COLS]

        cols = ID_COLS + [c for c in METRIC_COLS if c in g.columns] + param_cols_table
        gg = g[cols].copy()

        # Formateo de métricas a 6 decimales
        for c in ["c2_final_mae", "forget", "co2"]:
            if c in gg.columns:
                gg[c] = gg[c].apply(lambda x: _format_float(x, 6))

        # Construye la tabla en el docx
        table = doc.add_table(rows=1, cols=len(cols))
        table.style = "Light Grid" if "Light Grid" in [s.name for s in doc.styles] else table.style
        hdr_cells = table.rows[0].cells
        for j, c in enumerate(cols):
            hdr_cells[j].text = c

        for _, row in gg.iterrows():
            cells = table.add_row().cells
            for j, c in enumerate(cols):
                val = row[c]
                # evita "nan" como texto
                if pd.isna(val):
                    val = ""
                cells[j].text = str(val)

        # 3) Tendencias y correlaciones (Spearman sin SciPy)
        #    solo para columnas completamente numéricas
        num_cols = [c for c in param_cols_present if g[c].apply(_is_num).all()]
        # Tendencias Top-5
        if num_cols:
            doc.add_paragraph("Tendencias en el Top-5 (mediana [mín–máx]):")
            for c in num_cols:
                s = pd.to_numeric(g[c], errors="coerce")
                med = np.nanmedian(s); lo = np.nanmin(s); hi = np.nanmax(s)
                doc.add_paragraph(f"• {c} ≈ {med:.4g} [{lo:.4g}–{hi:.4g}]")
        # Correlaciones
        y_mae = pd.to_numeric(g["c2_final_mae"], errors="coerce") if "c2_final_mae" in g.columns else None
        y_for = pd.to_numeric(g["forget"], errors="coerce") if "forget" in g.columns else None
        any_corr = False
        lines_corr = []
        if (y_mae is not None) or (y_for is not None):
            for c in num_cols:
                x = pd.to_numeric(g[c], errors="coerce")
                r_mae = _spearman_safe(x, y_mae) if y_mae is not None else float("nan")
                r_for = _spearman_safe(x, y_for) if y_for is not None else float("nan")
                if not (math.isnan(r_mae) and math.isnan(r_for)):
                    any_corr = True
                    lines_corr.append(f"• {c} vs MAE: {r_mae:+.2f}  |  vs olvido: {r_for:+.2f}")
        if any_corr:
            doc.add_paragraph("Correlación (Spearman) con métricas:")
            for ln in lines_corr:
                doc.add_paragraph(ln)

        # Espacio entre métodos
        doc.add_paragraph("")

    doc.save(docx_path)
    print(f"[OK] Documento Word → {docx_path}")

except ImportError:
    # Fallback: no hay python-docx; dejamos guía clara
    print("[AVISO] python-docx no está instalado. Para generar el .docx, instala primero:")
    print("        pip install python-docx")
    # Como mínimo, exportamos un Markdown de respaldo para copy/paste:
    md_lines = [f"# Top-5 por método — {SUMMARY_DIR.name}", ""]
    for mb, g in top5.groupby("method_base", dropna=False):
        mb = str(mb or "unknown")
        g = _sort_group(g).reset_index(drop=True)
        md_lines.append(f"## {mb}")
        # tabla compacta (métricas + algunos parámetros)
        param_cols_present = [c for c in PARAM_COLS_ALL if c in g.columns and g[c].notna().any()]
        param_cols_present = sorted(param_cols_present, key=lambda c: g[c].notna().sum(), reverse=True)[:12]
        cols = ID_COLS + [c for c in METRIC_COLS if c in g.columns] + param_cols_present
        md_lines.append(g[cols].to_markdown(index=False))
        md_lines.append("")
    md_path = SUMMARY_DIR / "top5_por_metodo.md"
    Path(md_path).write_text("\n".join(md_lines), encoding="utf-8")
    print(f"[OK] Markdown de respaldo → {md_path}")


[OK] Documento Word → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_fast_rate_2025-11-16_1427/top5_por_metodo.docx
